In [ ]:
%matplotlib inline

import numpy as np
from pprint import pprint

from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
import torchvision
from torchvision import models, datasets, transforms
import torch.nn.functional as func
#torch.manual_seed(50)

import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"


print(torch.__version__, torchvision.__version__)
print (torch.cuda.get_device_name(device='cuda:0'))

In [ ]:
import os
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from zipfile import ZipFile
import zipfile

In [ ]:
import cv2
import os
import numpy as np
from PIL import Image 

In [ ]:
data_dir = 'Covid_chest_X-raydata\Covid_chest_X-raydata\Covid19-dataset'

print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

In [ ]:
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     #transforms.CenterCrop(32),
     transforms.Resize((32,32)),
     #transforms.RandomCrop(32),
     #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     #transforms.ConvertImageDtype(torch.float)
     ])

dataset = ImageFolder(data_dir+'/train', transform=transform)

#dataset1 = DataLoader(trainset, shuffle=True, batch_size=batch_size, num_workers=0)


In [ ]:
image_data=[]
target=[]
for i, j in dataset:
  image_data.append(i)
  target.append(j)

In [ ]:
#image_dataaaa=image_data[0:50]
#targetttt=target[0:50]

In [ ]:
image_data1=torch.stack(image_data)
image_data2=image_data1.numpy()
image_data2.shape

In [ ]:
qq=np.rollaxis(image_data2,1,4)
qq.shape

In [ ]:
# dst = datasets.CIFAR100("~/.torch", download=True)
# dst = datasets.MNIST("~/.torch", download=True)

tp = transforms.Compose([
    #transforms.Resize(32),
    #transforms.CenterCrop(32),
    transforms.ToTensor()
])
tt = transforms.ToPILImage()

device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
print("Running on %s" % device)

def label_to_onehot(target, num_classes=3):
    target = torch.unsqueeze(target, 1)
    onehot_target = torch.zeros(target.size(0), num_classes, device=target.device)
    onehot_target.scatter_(1, target, 1)
    return onehot_target

def cross_entropy_for_onehot(pred, target):
    return torch.mean(torch.sum(- target * F.log_softmax(pred, dim=-1), 1))

In [ ]:
# def weights_init(m):
#     if hasattr(m, "weight"):
#         m.weight.data.uniform_(-0.5, 0.5)
#         nn.init.xavier_uniform_(m.weight.data)
#     if hasattr(m, "bias"):
#         #m.bias.data.uniform_(-0.5, 0.5)
#         #nn.init.xavier_uniform(m.bias.data)
#         m.bias.data.fill_(0)



# class LeNet(nn.Module):

#     def __init__(self):

#         super(LeNet, self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, kernel_size=5,stride=2)
#         self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=2)
#         self.fc1 = nn.Linear(16*5*5, 256)
#         self.fc2 = nn.Linear(256, 120)
#         self.fc3 = nn.Linear(120, 106)

#     def forward(self, x):
#         #x = func.relu(self.conv1(x))
#         x = func.sigmoid(self.conv1(x))
#         #x = func.max_pool2d(x, 2)
#         #x = func.relu(self.conv2(x))
#         x = func.sigmoid(self.conv2(x))
#         #x = func.max_pool2d(x, 2)
#         x = x.view(x.size(0), -1)
#         #x = func.relu(self.fc1(x))
#         x = func.sigmoid(self.fc1(x))
#         #x = func.relu(self.fc2(x))
#         x = func.sigmoid(self.fc2(x))
#         x = self.fc3(x)
#         return x

    
    
# def weights_init(m):
#     if hasattr(m, "weight"):
#         m.weight.data.uniform_(-0.3, 0.3)
#     if hasattr(m, "bias"):
#         m.bias.data.uniform_(-0.3, 0.3)


def weights_init(m):
    if hasattr(m, "weight"):
        m.weight.data.uniform_(-0.5, 0.5)
    if hasattr(m, "bias"):
        m.bias.data.uniform_(-0.5, 0.5)


class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        act = nn.Sigmoid
        #act = nn.ReLU
        self.body = nn.Sequential(
            nn.Conv2d(3, 12, kernel_size=5, padding=5//2, stride=2),
            act(),
            nn.Conv2d(12, 12, kernel_size=5, padding=5//2, stride=2),
            act(),
            nn.Conv2d(12, 12, kernel_size=5, padding=5//2, stride=1),
            act(),
            nn.Conv2d(12, 12, kernel_size=5, padding=5//2, stride=1),
            act(),
        )
        self.fc = nn.Sequential(
            nn.Linear(768, 3)
        )
        
    def forward(self, x):
        out = self.body(x)
        out = out.view(out.size(0), -1)
        # print(out.size())
        out = self.fc(out)
        return out


net = LeNet().to(device)
net.apply(weights_init)



#criterion = cross_entropy_for_onehot
criterion = nn.CrossEntropyLoss()

In [ ]:
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Variable
from torch.utils import data

from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
#lfw_people=fetch_lfw_people(min_faces_per_person=14,color=True,slice_=(slice(61,189),slice(61,189)),resize=0.25) #14
#x=lfw_people.images
#y=lfw_people.target
x=qq
y=target

#target_names=lfw_people.target_names
n_classes=3

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,shuffle=True)



print (X_train.shape)
print (X_test.shape)
print (n_classes)


         

In [ ]:
test_size=len(X_test)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)
#X_train = torch.transpose
#X_train = X_train.astype('float32')
#X_train /= 255.0
#X_test /= 255.0

 


x_train = torch.FloatTensor(X_train).to(device)
x_train = x_train.transpose(2,3).transpose(1,2)
y_train = torch.LongTensor(y_train).to(device)

x_test = torch.FloatTensor(X_test).to(device)
x_test = x_test.transpose(2,3).transpose(1,2)
y_test = torch.LongTensor(y_test).to(device)


training = data.TensorDataset(x_train,y_train)

testing = data.TensorDataset(x_test,y_test)

dst_tensor=training

criterion_train = nn.CrossEntropyLoss()
optimizer_train = optim.Adam(net.parameters(),lr=0.01)#,momentum=0.9)
trainloader = torch.utils.data.DataLoader(training, batch_size=32, shuffle=False)



for epoch in range(0):

    for i,data in enumerate(trainloader,0):
    #for data in trainloader:
        #if i<=10: 

            inputs,label = data

            inputs,label =  Variable(inputs).to(device),Variable(label).to(device)

            optimizer_train.zero_grad()
            outputs_benign=net(inputs)

            loss_benign =  criterion_train(outputs_benign,label)

            loss_benign.backward()
            #sgd_update(net.parameters())

            optimizer_train.step()
            
            testloader = torch.utils.data.DataLoader(testing,batch_size=test_size, shuffle=False)

            acc =0.0
            for ji,tdata in enumerate(testloader,0):

                tinputs,tlabel = tdata

                tinputs,tlabel =  Variable(tinputs).to(device),Variable(tlabel).to(device)

                toutputs=net(tinputs)

                tpredict = torch.argmax(toutputs, dim=1)


                for mi in range(test_size):



                    if tpredict[mi] == tlabel[mi]:
                        acc=acc+1

            accuracy = acc / test_size
            print (accuracy)
            print ('fininshed testing')
            if accuracy>0.18:
                break


print ('fininshed training')
#torch.save(net.state_dict()

testloader = torch.utils.data.DataLoader(testing,batch_size=test_size, shuffle=False)

acc =0.0
for ji,tdata in enumerate(testloader,0):

    tinputs,tlabel = tdata

    tinputs,tlabel =  Variable(tinputs).to(device),Variable(tlabel).to(device)

    toutputs=net(tinputs)
    
    tpredict = torch.argmax(toutputs, dim=1)
    
   
    for mi in range(test_size):
        
        

        if tpredict[mi] == tlabel[mi]:
            acc=acc+1

accuracy = acc / test_size
print (accuracy)
print ('fininshed testing')

In [ ]:
plt.imshow(X_train[30], alpha =1)

In [ ]:
pip install pytorch-msssim

In [ ]:



######### honest partipant #########
img_index = 30   #use img_index
dst_pil = tt(dst_tensor[img_index][0].cpu())   #use img_index

gt_data = tp(dst_pil).to(device)
gt_data = torch.unsqueeze(gt_data,0)

gt_label = dst_tensor[img_index][1].long().to(device) #use img_index
gt_label = gt_label.view(1, )
gt_onehot_label = label_to_onehot(gt_label, num_classes=3)

plt.imshow(dst_pil)
#plt.savefig("./original/index_%s_label_%s"%(img_index,gt_label.item()))



batch =1  #
for bat in range(batch-1):
    dst_pil = tt(dst_tensor[img_index+1+bat][0].cpu())   #use img_index
    tmp = torch.unsqueeze(tp(dst_pil).to(device),0)
    #print(tmp.shape)
    gt_data = torch.cat((gt_data,tmp),0)

    gt_label_tmp = dst_tensor[img_index+1+bat][1].long().to(device) #use img_index
    gt_label_tmp = gt_label_tmp.view(1, )
    gt_label = torch.cat((gt_label,gt_label_tmp),0)
    gt_onehot_label = torch.cat((gt_onehot_label,label_to_onehot(gt_label_tmp, num_classes=3)),0)

    if gt_label_tmp ==60:
        print (bat)

    plt.imshow(dst_pil)
    #plt.savefig("./original/index_%s_label_%s"%(bat+1,gt_label_tmp.item()))

    #plt.title("Ground truth image")
    #print("GT label is %d." % gt_label.item(), "\nOnehot label is %d." % torch.argmax(gt_onehot_label, dim=-1).item())


gt_label = torch.reshape(gt_label,(-1,1))    
print (gt_data.shape)
print (gt_label.shape)
print (gt_label)
print (gt_onehot_label.shape)

plt.imshow(tt(gt_data[0].cpu()),cmap='gray')
# plt.axis('off')
# plt.savefig("./attack_image/tifs")


# compute original gradient 
dy_dx = []
original_dy_dx=[]
original_pred = []
for item in range(batch):
    gt_data_single = torch.unsqueeze(gt_data[item],0)
    out = net(gt_data_single)
    #y = criterion(out, gt_onehot_label[item])
    y = criterion(out, gt_label[item])
    dy_dx = torch.autograd.grad(y, net.parameters(),retain_graph=True)
    original_dy_dx_tmp = list((_.detach().clone() for _ in dy_dx))
    original_dy_dx.append(original_dy_dx_tmp)
    out_tmp = out.detach().clone()
    original_pred.append(out_tmp)





# #FOR fully-connected model only
#     dw = net.body[0].weight
#     db = net.body[0].bias
#     dy_dw = torch.autograd.grad(y, dw,retain_graph=True)
#     dy_db = torch.autograd.grad(y, db,retain_graph=True)

#     print (dy_dw)
#     #print (dy_db.shape)

#     leak=dy_dw/dy_db

#     print (leak.shape)



# share the gradients with other clients
#original_dy_dx = list((_.detach().clone() for _ in dy_dx))


# generate dummy data and label

import time

from pytorch_msssim import ssim


#print (ssim(0.43*torch.unsqueeze(gt_data[0],dim=0),torch.unsqueeze(gt_data[0],dim=0),data_range=0).item())
#print (torch.dist(0.6*torch.unsqueeze(gt_data[0],dim=0),torch.unsqueeze(gt_data[0],dim=0),2).item())


for item in range(1):
    start = time.process_time()
    for rd in range(1):

        torch.manual_seed(200*rd)
        #dummy_data = torch.unsqueeze(torch.randn(gt_data[item].size()),0).to(device).requires_grad_(True)

        #dummy_data = torch.unsqueeze(torch.zeros(gt_data[item].size()),0).to(device).requires_grad_(True)
        #dummy_data = torch.unsqueeze(torch.ones(gt_data[item].size()),0).to(device).requires_grad_(True)


        #background = torch.unsqueeze(torch.zeros(gt_data[item].size()),0)
        #background[0,0,::] = 1
        #dummy_data = background.to(device).requires_grad_(True)
        ##dummy_data = (torch.unsqueeze(torch.randn(gt_data[item].size()),0)+background).to(device).requires_grad_(True)

        #surrogate = torch.unsqueeze(gt_data[item+1],0)
        #aaa = torch.rand([3,16,16])
        #surrogate[0,:,8:24,8:24] =aaa
        #dummy_data = surrogate.to(device).requires_grad_(True)    

        #dummy_data = torch.unsqueeze(gt_data[item+1],0).to(device).requires_grad_(True)

        #k = np.random.randint(0,95)
        #dummy_data = torch.unsqueeze(gt_data[k],0).to(device).requires_grad_(True)


        pat_1 = torch.rand([3,16,16])
        pat_2 = torch.cat((pat_1,pat_1),dim=1)
        pat_4 = torch.cat((pat_2,pat_2),dim=2)
        dummy_data = torch.unsqueeze(pat_4,dim=0).to(device).requires_grad_(True)


        #aaa = torch.rand([3,8,8])
        #bbb = torch.cat((aaa,aaa),dim=1)
        #ccc = torch.cat((bbb,bbb),dim=1)
        #ddd = torch.cat((ccc,ccc),dim=2)
        #eee = torch.cat((ddd,ddd),dim=2)
        #dummy_data = torch.unsqueeze(eee,dim=0).to(device).requires_grad_(True)

        #aaa = torch.rand([3,4,4])
        #bbb = torch.cat((aaa,aaa),dim=1)
        #ccc = torch.cat((bbb,bbb),dim=1)
        #ddd = torch.cat((ccc,ccc),dim=1)
        #eee = torch.cat((ddd,ddd),dim=2)
        #fff = torch.cat((eee,eee),dim=2)
        #ggg = torch.cat((fff,fff),dim=2)
        #dummy_data = torch.unsqueeze(ggg,dim=0).to(device).requires_grad_(True)


        #dummy_data = plt.imread("./attack_image/replacement_69.png")
        #print (dummy_data.shape)
        #dummy_data = torch.FloatTensor(dummy_data).to(device)
        #dummy_data = dummy_data.transpose(2,3).transpose(1,2)

        dummy_unsqueeze=torch.unsqueeze(gt_onehot_label[item],dim=0)

        dummy_label = torch.randn(dummy_unsqueeze.size()).to(device).requires_grad_(True)
        label_pred=torch.argmin(torch.sum(original_dy_dx[item][-2], dim=-1), 
                                dim=-1).detach().reshape((1,)).requires_grad_(False)
        #print (original_dy_dx[item][-1].shape)
        #print (original_dy_dx[item][-1].argmin())

        #print (torch.sum(original_dy_dx[item][-2], dim=-1).argmin())

        plt.imshow(tt(dummy_data[0].cpu()))
        plt.title("Dummy data")
        #plt.savefig("./random_seed/index_%s_rand_seed_%s_label_%s"%(item,rd,torch.argmax(dummy_label, dim=-1).item()))

        plt.clf()
        print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
        print("stolen label is %d." % label_pred.item())


        #optimizer = torch.optim.LBFGS([dummy_data,dummy_label])
        optimizer = torch.optim.LBFGS([dummy_data,])
        #optimizer = torch.optim.AdamW([dummy_data,],lr=0.01)
        #optimizer = torch.optim.SGD([dummy_data,],lr=0.01)



        history = []

        percept_dis = np.zeros(300)
        recover_dis = np.zeros(300)
        for iters in range(300):


            percept_dis[iters]=ssim(dummy_data,torch.unsqueeze(gt_data[item],dim=0),data_range=0).item()
            #recover_dis[iters]=torch.dist(dummy_data,torch.unsqueeze(gt_data[item],dim=0),2).item()
            recover_dis[iters]= F.mse_loss(dummy_data,torch.unsqueeze(gt_data[item],dim=0)).item()

            history.append(tt(dummy_data[0].cpu()))
            def closure():
                optimizer.zero_grad()

                pred = net(dummy_data) 

                #dummy_onehot_label = F.softmax(dummy_label, dim=-1).long()

                #dummy_loss = criterion(pred, dummy_onehot_label) # TODO: fix the gt_label to dummy_label in both code and slides.
                ##print (pred)
                ##print (label_pred)

                dummy_loss = criterion(pred, label_pred)
                dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)
                ##dummy_dy_dp = torch.autograd.grad(dummy_loss, dummy_data, create_graph=True)
                ##print (dummy_dy_dp[0].shape)  

                grad_diff = 0
                grad_count = 0
                #count =0
                for gx, gy in zip(dummy_dy_dx, original_dy_dx[item]): # TODO: fix the variablas here

                    #if iters==500 or iters== 1200:
                    #print (gx[0])
                    #    print ('hahaha')
                    #print (gy[0])
                    lasso = torch.norm(dummy_data,p=1)
                    ridge = torch.norm(dummy_data,p=2)
                    grad_diff += ((gx - gy) ** 2).sum() #+ 0.0*lasso +0.01*ridge 

                    #print (gx.shape)

                    grad_count += gx.nelement()


                    #if count == 9:
                    #    break
                    #count=count+1
                # grad_diff = grad_diff / grad_count * 1000

                #grad_diff += ((original_pred[item]-pred)**2).sum()




                grad_diff.backward()
                #print (count)

                #print (dummy_dy_dx)
                #print (original_dy_dx)


                return grad_diff



            optimizer.step(closure)
            if iters % 5 == 0: 
                current_loss = closure()
                #if iters == 0: 
                print ("%.8f" % current_loss.item())
                #print(iters, "%.8f" % current_loss.item())
            history.append(tt(dummy_data[0].cpu()))



        #plt.figure(figsize=(18, 12))
        #for i in range(60):
        #  plt.subplot(6, 10, i + 1)
        #  plt.imshow(history[i * 5])
        #  plt.title("iter=%d" % (i * 5))
        #  plt.axis('off')

        plt.figure(figsize=(12, 1.5))
        #iter_idx = [0,20,40,60,80,100,120,140,160,180]
        plt.figure(figsize=(6.5, 1.2))
        #iter_idx = [0,1000,2000,3000,4000,5000]
        iter_idx = [0,5,10,20,50,100]


        for i in range(6):
          plt.subplot(1, 6, i + 1)
          plt.imshow(history[iter_idx[i]])
          plt.title("iter=%d" % (iter_idx[i]))
          plt.axis('off')

        #np.savetxt('ssim_random2',percept_dis,fmt="%4f")
        #np.savetxt('mse_random2',recover_dis,fmt="%4f")

        #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
        #plt.savefig("./attack_image/index_%s_rand_%s_label_%s"%(item,rd, label_pred.item()))
        #plt.clf()

    duration = time.process_time()-start
    #print ("Running time is %.4f." %(duration/10.0) )
    print (duration/10.0 )



In [ ]:
gau_rate_list = [100]
for gau_rate in gau_rate_list:
    print('now starting',gau_rate)


    ######### honest partipant #########
    img_index = 30   #use img_index
    dst_pil = tt(dst_tensor[img_index][0].cpu())   #use img_index

    gt_data = tp(dst_pil).to(device)
    gt_data = torch.unsqueeze(gt_data,0)

    gt_label = dst_tensor[img_index][1].long().to(device) #use img_index
    gt_label = gt_label.view(1, )
    gt_onehot_label = label_to_onehot(gt_label, num_classes=3)

    plt.imshow(dst_pil)
    #plt.savefig("./original/index_%s_label_%s"%(img_index,gt_label.item()))



    batch =1  #
    for bat in range(batch-1):
        dst_pil = tt(dst_tensor[img_index+1+bat][0].cpu())   #use img_index
        tmp = torch.unsqueeze(tp(dst_pil).to(device),0)
        #print(tmp.shape)
        gt_data = torch.cat((gt_data,tmp),0)

        gt_label_tmp = dst_tensor[img_index+1+bat][1].long().to(device) #use img_index
        gt_label_tmp = gt_label_tmp.view(1, )
        gt_label = torch.cat((gt_label,gt_label_tmp),0)
        gt_onehot_label = torch.cat((gt_onehot_label,label_to_onehot(gt_label_tmp, num_classes=3)),0)

        if gt_label_tmp ==60:
            print (bat)

        plt.imshow(dst_pil)
        #plt.savefig("./original/index_%s_label_%s"%(bat+1,gt_label_tmp.item()))

        #plt.title("Ground truth image")
        #print("GT label is %d." % gt_label.item(), "\nOnehot label is %d." % torch.argmax(gt_onehot_label, dim=-1).item())


    gt_label = torch.reshape(gt_label,(-1,1))    
    print (gt_data.shape)
    print (gt_label.shape)
    print (gt_label)
    print (gt_onehot_label.shape)

    plt.imshow(tt(gt_data[0].cpu()),cmap='gray')
    # plt.axis('off')
    # plt.savefig("./attack_image/tifs")


    # compute original gradient 
    dy_dx = []
    original_dy_dx=[]
    original_pred = []
    for item in range(batch):
        gt_data_single = torch.unsqueeze(gt_data[item],0)
        out = net(gt_data_single)
        #y = criterion(out, gt_onehot_label[item])
        y = criterion(out, gt_label[item])
        dy_dx = torch.autograd.grad(y, net.parameters(),retain_graph=True)
        original_dy_dx_tmp = list((_.detach().clone() for _ in dy_dx))
        original_dy_dx.append(original_dy_dx_tmp)
        out_tmp = out.detach().clone()
        original_pred.append(out_tmp)

        
    #if gaussian noise or laplace
        m = torch.distributions.laplace.Laplace(torch.tensor([0.0]), torch.tensor([1/gau_rate]))
        for item in range(batch):
            for layer_idx in range(10):
                #original_dy_dx[item][layer_idx] =  original_dy_dx[item][layer_idx] + torch.empty(original_dy_dx[item][layer_idx].size()).normal_(mean=0,std=1/gau_rate).to(device)
                original_dy_dx[item][layer_idx] =  original_dy_dx[item][layer_idx] + torch.squeeze(m.sample(sample_shape=original_dy_dx[item][layer_idx].size()),dim=-1).to(device)
        #dy_dx.append(torch.autograd.grad(y, net.parameters()))



    # #FOR fully-connected model only
    #     dw = net.body[0].weight
    #     db = net.body[0].bias
    #     dy_dw = torch.autograd.grad(y, dw,retain_graph=True)
    #     dy_db = torch.autograd.grad(y, db,retain_graph=True)

    #     print (dy_dw)
    #     #print (dy_db.shape)

    #     leak=dy_dw/dy_db

    #     print (leak.shape)



    # share the gradients with other clients
    #original_dy_dx = list((_.detach().clone() for _ in dy_dx))


    # generate dummy data and label

    import time

    from pytorch_msssim import ssim


    #print (ssim(0.43*torch.unsqueeze(gt_data[0],dim=0),torch.unsqueeze(gt_data[0],dim=0),data_range=0).item())
    #print (torch.dist(0.6*torch.unsqueeze(gt_data[0],dim=0),torch.unsqueeze(gt_data[0],dim=0),2).item())


    for item in range(1):
        start = time.process_time()
        for rd in range(1):

            torch.manual_seed(200*rd)
            #dummy_data = torch.unsqueeze(torch.randn(gt_data[item].size()),0).to(device).requires_grad_(True)

            #dummy_data = torch.unsqueeze(torch.zeros(gt_data[item].size()),0).to(device).requires_grad_(True)
            #dummy_data = torch.unsqueeze(torch.ones(gt_data[item].size()),0).to(device).requires_grad_(True)


            #background = torch.unsqueeze(torch.zeros(gt_data[item].size()),0)
            #background[0,0,::] = 1
            #dummy_data = background.to(device).requires_grad_(True)
            ##dummy_data = (torch.unsqueeze(torch.randn(gt_data[item].size()),0)+background).to(device).requires_grad_(True)

            #surrogate = torch.unsqueeze(gt_data[item+1],0)
            #aaa = torch.rand([3,16,16])
            #surrogate[0,:,8:24,8:24] =aaa
            #dummy_data = surrogate.to(device).requires_grad_(True)    

            #dummy_data = torch.unsqueeze(gt_data[item+1],0).to(device).requires_grad_(True)

            #k = np.random.randint(0,95)
            #dummy_data = torch.unsqueeze(gt_data[k],0).to(device).requires_grad_(True)


            pat_1 = torch.rand([3,16,16])
            pat_2 = torch.cat((pat_1,pat_1),dim=1)
            pat_4 = torch.cat((pat_2,pat_2),dim=2)
            dummy_data = torch.unsqueeze(pat_4,dim=0).to(device).requires_grad_(True)


            #aaa = torch.rand([3,8,8])
            #bbb = torch.cat((aaa,aaa),dim=1)
            #ccc = torch.cat((bbb,bbb),dim=1)
            #ddd = torch.cat((ccc,ccc),dim=2)
            #eee = torch.cat((ddd,ddd),dim=2)
            #dummy_data = torch.unsqueeze(eee,dim=0).to(device).requires_grad_(True)

            #aaa = torch.rand([3,4,4])
            #bbb = torch.cat((aaa,aaa),dim=1)
            #ccc = torch.cat((bbb,bbb),dim=1)
            #ddd = torch.cat((ccc,ccc),dim=1)
            #eee = torch.cat((ddd,ddd),dim=2)
            #fff = torch.cat((eee,eee),dim=2)
            #ggg = torch.cat((fff,fff),dim=2)
            #dummy_data = torch.unsqueeze(ggg,dim=0).to(device).requires_grad_(True)


            #dummy_data = plt.imread("./attack_image/replacement_69.png")
            #print (dummy_data.shape)
            #dummy_data = torch.FloatTensor(dummy_data).to(device)
            #dummy_data = dummy_data.transpose(2,3).transpose(1,2)

            dummy_unsqueeze=torch.unsqueeze(gt_onehot_label[item],dim=0)

            dummy_label = torch.randn(dummy_unsqueeze.size()).to(device).requires_grad_(True)
            label_pred=torch.argmin(torch.sum(original_dy_dx[item][-2], dim=-1), 
                                    dim=-1).detach().reshape((1,)).requires_grad_(False)
            #print (original_dy_dx[item][-1].shape)
            #print (original_dy_dx[item][-1].argmin())

            #print (torch.sum(original_dy_dx[item][-2], dim=-1).argmin())

            plt.imshow(tt(dummy_data[0].cpu()))
            plt.title("Dummy data")
            #plt.savefig("./random_seed/index_%s_rand_seed_%s_label_%s"%(item,rd,torch.argmax(dummy_label, dim=-1).item()))

            plt.clf()
            print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
            print("stolen label is %d." % label_pred.item())


            #optimizer = torch.optim.LBFGS([dummy_data,dummy_label])
            optimizer = torch.optim.LBFGS([dummy_data,])
            #optimizer = torch.optim.AdamW([dummy_data,],lr=0.01)
            #optimizer = torch.optim.SGD([dummy_data,],lr=0.01)



            history = []

            percept_dis = np.zeros(300)
            recover_dis = np.zeros(300)
            for iters in range(300):


                percept_dis[iters]=ssim(dummy_data,torch.unsqueeze(gt_data[item],dim=0),data_range=0).item()
                #recover_dis[iters]=torch.dist(dummy_data,torch.unsqueeze(gt_data[item],dim=0),2).item()
                recover_dis[iters]= F.mse_loss(dummy_data,torch.unsqueeze(gt_data[item],dim=0)).item()

                history.append(tt(dummy_data[0].cpu()))
                def closure():
                    optimizer.zero_grad()

                    pred = net(dummy_data) 

                    #dummy_onehot_label = F.softmax(dummy_label, dim=-1).long()

                    #dummy_loss = criterion(pred, dummy_onehot_label) # TODO: fix the gt_label to dummy_label in both code and slides.
                    ##print (pred)
                    ##print (label_pred)

                    dummy_loss = criterion(pred, label_pred)
                    dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)
                    ##dummy_dy_dp = torch.autograd.grad(dummy_loss, dummy_data, create_graph=True)
                    ##print (dummy_dy_dp[0].shape)  

                    grad_diff = 0
                    grad_count = 0
                    #count =0
                    for gx, gy in zip(dummy_dy_dx, original_dy_dx[item]): # TODO: fix the variablas here

                        #if iters==500 or iters== 1200:
                        #print (gx[0])
                        #    print ('hahaha')
                        #print (gy[0])
                        lasso = torch.norm(dummy_data,p=1)
                        ridge = torch.norm(dummy_data,p=2)
                        grad_diff += ((gx - gy) ** 2).sum() #+ 0.0*lasso +0.01*ridge 

                        #print (gx.shape)

                        grad_count += gx.nelement()


                        #if count == 9:
                        #    break
                        #count=count+1
                    # grad_diff = grad_diff / grad_count * 1000

                    #grad_diff += ((original_pred[item]-pred)**2).sum()




                    grad_diff.backward()
                    #print (count)

                    #print (dummy_dy_dx)
                    #print (original_dy_dx)


                    return grad_diff



                optimizer.step(closure)
                if iters % 5 == 0: 
                    current_loss = closure()
                    #if iters == 0: 
                    print ("%.8f" % current_loss.item())
                    #print(iters, "%.8f" % current_loss.item())
                history.append(tt(dummy_data[0].cpu()))



            #plt.figure(figsize=(18, 12))
            #for i in range(60):
            #  plt.subplot(6, 10, i + 1)
            #  plt.imshow(history[i * 5])
            #  plt.title("iter=%d" % (i * 5))
            #  plt.axis('off')

            plt.figure(figsize=(12, 1.5))
            #iter_idx = [0,20,40,60,80,100,120,140,160,180]
            plt.figure(figsize=(6.5, 1.2))
            #iter_idx = [0,1000,2000,3000,4000,5000]
            iter_idx = [0,5,10,20,50,100]


            for i in range(6):
              plt.subplot(1, 6, i + 1)
              plt.imshow(history[iter_idx[i]])
              plt.title("iter=%d" % (iter_idx[i]))
              plt.axis('off')

            #np.savetxt('ssim_random2',percept_dis,fmt="%4f")
            #np.savetxt('mse_random2',recover_dis,fmt="%4f")

            #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
            #plt.savefig("./attack_image/index_%s_rand_%s_label_%s"%(item,rd, label_pred.item()))
            #plt.clf()

        duration = time.process_time()-start
        #print ("Running time is %.4f." %(duration/10.0) )
        print (duration/10.0 )



In [ ]:
percept_dis[299]

In [ ]:
recover_dis[299]

In [ ]:
plt.figure(figsize=(12, 8))
for i in range(60):
  plt.subplot(6, 10, i + 1)
  plt.imshow(history[i * 5])
  plt.title("iter=%d" % (i * 5))
  plt.axis('off')
print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())

# Noise Level 100

In [ ]:
# from pytorch_msssim import ssim

#dgc_rate_list = [20,30,40,50,70, 80,90]

#for epoch in range(1):
#for dgc_rate in dgc_rate_list:
gau_rate_list = [100]
for gau_rate in gau_rate_list:

    
    print('now starting',gau_rate)

    for iter_,data in enumerate(trainloader,1):
        
        if iter_ != 1:
            break
   
        ######### honest partipant #########
        img_index = 30   #use img_index
        dst_pil = tt(dst_tensor[img_index][0].cpu())   #use img_index

        gt_data = transform(dst_pil).to(device)
        gt_data = torch.unsqueeze(gt_data,0)

        gt_label = dst_tensor[img_index][1].long().to(device) #use img_index
        gt_label = gt_label.view(1, )
        gt_onehot_label = label_to_onehot(gt_label, num_classes=3)

        #plt.imshow(dst_pil)
        #plt.axis('off')
        plt.savefig("sample_data_COVID_Xray.png")



        batch =2  #
        for bat in range(batch-1):
            dst_pil = tt(dst_tensor[img_index+1+bat][0].cpu())   #use img_index
            tmp = torch.unsqueeze(transform(dst_pil).to(device),0)
            #print(tmp.shape)
            gt_data = torch.cat((gt_data,tmp),0)

            gt_label_tmp = dst_tensor[img_index+1+bat][1].long().to(device) #use img_index
            gt_label_tmp = gt_label_tmp.view(1, )
            gt_label = torch.cat((gt_label,gt_label_tmp),0)
            gt_onehot_label = torch.cat((gt_onehot_label,label_to_onehot(gt_label_tmp, num_classes=3)),0)

            plt.imshow(dst_pil)
            #plt.savefig("./original/index_%s_label_%s"%(bat+1,gt_label_tmp.item()))

            #plt.title("Ground truth image")
            #print("GT label is %d." % gt_label.item(), "\nOnehot label is %d." % torch.argmax(gt_onehot_label, dim=-1).item())


        gt_label = torch.reshape(gt_label,(-1,1))    
        #print (gt_data.shape)
        #print (gt_label.shape)
        #print (gt_onehot_label.shape)


        # compute original gradient 
        dy_dx = []
        original_dy_dx=[]
        original_pred = []
        for item in range(batch):
            gt_data_single = torch.unsqueeze(gt_data[item],0)
            out = net(gt_data_single)
            #y = criterion(out, gt_onehot_label[item])
            y = criterion(out, gt_label[item])
            dy_dx = torch.autograd.grad(y, net.parameters(),retain_graph=True)
            original_dy_dx_tmp = list((_.detach().clone() for _ in dy_dx))
            original_dy_dx.append(original_dy_dx_tmp)
            out_tmp = out.detach().clone()
            original_pred.append(out_tmp)
            
            
        #if gaussian noise or laplace
        m = torch.distributions.laplace.Laplace(torch.tensor([0.0]), torch.tensor([1/gau_rate]))
        for item in range(batch):
            for layer_idx in range(10):
                #original_dy_dx[item][layer_idx] =  original_dy_dx[item][layer_idx] + torch.empty(original_dy_dx[item][layer_idx].size()).normal_(mean=0,std=1/gau_rate).to(device)
                original_dy_dx[item][layer_idx] =  original_dy_dx[item][layer_idx] + torch.squeeze(m.sample(sample_shape=original_dy_dx[item][layer_idx].size()),dim=-1).to(device)
                #break
        ##if deep gradient compression
        #print (original_dy_dx[0][0][0])
        #for item in range(batch):
        #    for layer_idx in range(10):
        #        if layer_idx == 0:    
        #            flat_dy_dx = torch.flatten(original_dy_dx[item][layer_idx])
        #        else:
        #            flat_dy_dx = torch.cat((flat_dy_dx,torch.flatten(original_dy_dx[item][layer_idx])),0)
        #sorted_dy_dx = flat_dy_dx.abs().sort()
        #size = np.asarray(list(flat_dy_dx.shape))
        #thresh = sorted_dy_dx[0][int(size * dgc_rate/100.0)]
        #print (size)
        #print (int(size * dgc_rate/100.0))
        #print (thresh)
        #print (sorted_dy_dx[0][-1])
        #for item in range(batch):
        #    for layer_idx in range(10):
        #        shape_tmp = original_dy_dx[item][layer_idx].size()
        #        flat_dy_dx_prune = torch.flatten(original_dy_dx[item][layer_idx])
        #        size_tmp = np.asarray(list(flat_dy_dx_prune.shape))
        #        for m in range(int(size_tmp)):
        #            if flat_dy_dx_prune[m].abs()<=thresh:
        #                flat_dy_dx_prune[m] = 0
        #        original_dy_dx[item][layer_idx] = flat_dy_dx_prune.view(shape_tmp)
        #print (original_dy_dx[0][0][0])
              

        # generate dummy data and label
        import time

        #if iter_ % 10 ==0: 
        if iter_ == 1:
            
            #print ('epoch',epoch,'iter',iter_)
            for item in range(1):
                start = time.process_time()
                for rd in range(1):

                    torch.manual_seed(100*rd)

                    pat_1 = torch.rand([3,16,16])
                    pat_2 = torch.cat((pat_1,pat_1),dim=1)
                    pat_4 = torch.cat((pat_2,pat_2),dim=2)
                    dummy_data = torch.unsqueeze(pat_4,dim=0).to(device).requires_grad_(True)     

                    dummy_unsqueeze=torch.unsqueeze(gt_onehot_label[item],dim=0)

                    dummy_label = torch.randn(gt_onehot_label[item].size()).to(device).requires_grad_(True)
                    label_pred = torch.argmin(torch.sum(original_dy_dx[item][-2], dim=-1), dim=-1).detach().reshape((1,)).requires_grad_(False)
                    label_pred_onehot = label_to_onehot(label_pred, num_classes=3)

                    plt.imshow(tt(dummy_data[0].cpu()))
                    plt.title("Dummy data")
                    #plt.savefig("./random_seed/index_%s_rand_seed_%s_label_%s"%(item,rd,torch.argmax(dummy_label, dim=-1).item()))

                    plt.clf()
                    #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
                    #print("stolen label is %d." % label_pred.item())

                    #optimizer = torch.optim.LBFGS([dummy_data,dummy_label])
                    optimizer = torch.optim.LBFGS([dummy_data,])


                    history = []
                    history_batch = []
                    history_grad = []

                    percept_dis = np.zeros(100)
                    recover_dis = np.zeros(100)
                    for iters in range(100):

                        percept_dis[iters]=ssim(dummy_data,torch.unsqueeze(gt_data[item],dim=0),data_range=0).item()
                        recover_dis[iters]=torch.dist(dummy_data,torch.unsqueeze(gt_data[item],dim=0),2).item()

                        history.append(tt(dummy_data[0].cpu()))

                        def closure():
                            optimizer.zero_grad()

                            pred = net(dummy_data) 
                            dummy_onehot_label = F.softmax(dummy_label, dim=-1)
                            #dummy_loss = criterion(pred, dummy_onehot_label) # TODO: fix the gt_label to dummy_label in both code and slides.

                            #dummy_loss = criterion(pred, label_pred_onehot)
                            dummy_loss = criterion(pred, label_pred)


                            dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)
                            #dummy_dy_dp = torch.autograd.grad(dummy_loss, dummy_data, create_graph=True)
                            #print (dummy_dy_dp[0].shape)

                            grad_diff = 0
                            grad_count = 0
                            #count =0
                            for gx, gy in zip(dummy_dy_dx, original_dy_dx[item]): # TODO: fix the variablas here

                                #if iters==500 or iters== 1200:
                                #    print (gx[0])
                                #    print ('hahaha')
                                #    print (gy[0])
                                lasso = torch.norm(dummy_data,p=1)
                                ridge = torch.norm(dummy_data,p=2)
                                grad_diff += ((gx - gy) ** 2).sum() #+ 0.0*lasso +0.01*ridge


                                grad_count += gx.nelement()

                                #if count == 9:
                                #    break
                                #count=count+1
                            # grad_diff = grad_diff / grad_count * 1000
                            grad_diff.backward()
                            #print (count)

                            #print (dummy_dy_dx)
                            #print (original_dy_dx)


                            return grad_diff



                        optimizer.step(closure)
                        if iters % 5 == 0: 
                            current_loss = closure()
                            #if iters == 0: 
                            #print ("%.8f" % current_loss.item())
                            #print(iters, "%.8f" % current_loss.item())

                    #     for bat in range(batch-1):
                    #         history_batch.append(tt(dummy_data[bat].cpu()))

                    #plt.figure(figsize=(30, 20))
                    #for i in range(100):
                    #    plt.subplot(10, 10, i + 1)
                    #    plt.imshow(history[i * 5])
                    #    plt.title("iter=%d" % (i * 5))
                    #    plt.axis('off')
                    #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())

                    #np.savetxt('./attack_image/lfw_ssim_idx_%s_laplace_%s'%(item,gau_rate),percept_dis,fmt="%4f")
                    #np.savetxt('./attack_image/lfw_mse_idx_%s_laplace_%s'%(item,gau_rate),recover_dis,fmt="%4f")
                    #plt.savefig("./attack_image/lfw_index_%s_laplace_%s"%(item,gau_rate))

                    #plt.clf()
                    
                    pinp = np.argmin(recover_dis)
                    plt.imshow(history[pinp])
                    plt.axis('off')
                    
                    #plt.figure(figsize=(15, 10))
                    #for i in range(60):
                    #    plt.subplot(6, 10, i + 1)
                    #    plt.imshow(history[i * 14 ], cmap='gray')
                    #    plt.title("iter=%d" % (i*14))
                    #    plt.axis('off')
                    #plt.savefig("/content/sample_data"%(item,gau_rate))
                    
                #duration = time.clock()-start
                #print ("Running time is %.4f." %(duration/10.0) )
                #print (duration)
                
        
                
                
                
                
################################################### training when set to epoch

#         #if epoch>=1:
#         #if i==1:
#             #break
#         #print (iter_)
#         inputs,label = data

#         inputs,label =  Variable(inputs).to(device),Variable(label).to(device)

#         optimizer_train.zero_grad()


#         outputs_benign=net(inputs)
#         #outputs_benign = F.softmax(outputs_benign, dim=-1)
#         #print (outputs_benign[0])


#         loss_benign =  criterion_train(outputs_benign,label)

#         #print("loss computed")
#         loss_benign.backward()
#         #print("loss BP")
#         optimizer_train.step()
#         #sgd_update(net.parameters())

#         #if i%2000==0:
#         #print (loss_benign.item())
#         #torch.save(net.state_dict(),'./LFW_net.pth')  

#         #if  iter_%50==0:
#         #    print ('attack',iter_)
       
        
#         print ('fininshed training')
#         break
###############################   testing
    
#         total = len(y_test)
#         acc =0.0
#         for ct in range(total):
#             testing_data = tt(testing[ct][0].cpu())
#             testing_data1 = tp(testing_data).to(device)
#             testing_data2 = testing_data1.view(1, *testing_data1.size())
#             y_pred = net(testing_data2)
#             predicted = torch.argmax(y_pred)

#             if predicted == y_test[ct]:
#                 acc=acc+1
#         accuracy = acc / total
#         print (accuracy)
#         print ('fininshed testing')



In [ ]:
from pytorch_msssim import ssim


for epoch in range(1):

    for iter_,data in enumerate(trainloader,1):
   
        ######### honest partipant #########
        img_index = 30   #use img_index
        dst_pil = tt(dst_tensor[img_index][0].cpu())   #use img_index

        gt_data = transform(dst_pil).to(device)
        gt_data = torch.unsqueeze(gt_data,0)

        gt_label = dst_tensor[img_index][1].long().to(device) #use img_index
        gt_label = gt_label.view(1, )
        gt_onehot_label = label_to_onehot(gt_label, num_classes=3)

        plt.imshow(dst_pil)
        #plt.savefig("./original/index_%s_label_%s"%(img_index,gt_label.item()))



        batch =32  #
        for bat in range(batch-1):
            dst_pil = tt(dst_tensor[img_index+1+bat][0].cpu())   #use img_index
            tmp = torch.unsqueeze(transform(dst_pil).to(device),0)
            #print(tmp.shape)
            gt_data = torch.cat((gt_data,tmp),0)

            gt_label_tmp = dst_tensor[img_index+1+bat][1].long().to(device) #use img_index
            gt_label_tmp = gt_label_tmp.view(1, )
            gt_label = torch.cat((gt_label,gt_label_tmp),0)
            gt_onehot_label = torch.cat((gt_onehot_label,label_to_onehot(gt_label_tmp, num_classes=3)),0)

            plt.imshow(dst_pil)
            #plt.savefig("./original/index_%s_label_%s"%(bat+1,gt_label_tmp.item()))

            #plt.title("Ground truth image")
            #print("GT label is %d." % gt_label.item(), "\nOnehot label is %d." % torch.argmax(gt_onehot_label, dim=-1).item())


        gt_label = torch.reshape(gt_label,(-1,1))    
        print ("Image Shape = ",gt_data.shape)
        print ("Label shape = ",gt_label.shape)
        print ("Onehot Label shape = ",gt_onehot_label.shape)


        # compute original gradient 
        dy_dx = []
        original_dy_dx=[]
        original_pred = []
        for item in range(batch):
            gt_data_single = torch.unsqueeze(gt_data[item],0)
            out = net(gt_data_single)
            #y = criterion(out, gt_onehot_label[item])
            y = criterion(out, gt_label[item])
            dy_dx = torch.autograd.grad(y, net.parameters(),retain_graph=True)
            original_dy_dx_tmp = list((_.detach().clone() for _ in dy_dx))
            original_dy_dx.append(original_dy_dx_tmp)
            out_tmp = out.detach().clone()
            original_pred.append(out_tmp)

            #dy_dx.append(torch.autograd.grad(y, net.parameters()))





        # share the gradients with other clients
        #original_dy_dx = list((_.detach().clone() for _ in dy_dx))


        # generate dummy data and label
        import time


        for item in range(1):
            start = time.process_time()
            for rd in range(1):

                torch.manual_seed(100*rd)
        
                pat_1 = torch.rand([3,16,16])
                pat_2 = torch.cat((pat_1,pat_1),dim=1)
                pat_4 = torch.cat((pat_2,pat_2),dim=2)
                dummy_data = torch.unsqueeze(pat_4,dim=0).to(device).requires_grad_(True)     

                dummy_unsqueeze=torch.unsqueeze(gt_onehot_label[item],dim=0)

                dummy_label = torch.randn(dummy_unsqueeze.size()).to(device).requires_grad_(True)
                label_pred=torch.argmin(torch.sum(original_dy_dx[item][-2], dim=-1), 
                                        dim=-1).detach().reshape((1,)).requires_grad_(False)
                #print (original_dy_dx[item][-1].shape)
                #print (original_dy_dx[item][-1].argmin())

                #print (torch.sum(original_dy_dx[item][-2], dim=-1).argmin())

                plt.imshow(tt(dummy_data[0].cpu()))
                plt.title("Dummy data")
                #plt.savefig("./random_seed/index_%s_rand_seed_%s_label_%s"%(item,rd,torch.argmax(dummy_label, dim=-1).item()))

                plt.clf()
                print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
                print("stolen label is %d." % label_pred.item())


                #optimizer = torch.optim.LBFGS([dummy_data,dummy_label])
                optimizer = torch.optim.LBFGS([dummy_data,])
                #optimizer = torch.optim.AdamW([dummy_data,],lr=0.01)


                history = []
                history_batch = []
                history_grad = []
                
                percept_dis = np.zeros(500)
                recover_dis = np.zeros(500)
                for iters in range(500):
                    
                    percept_dis[iters]=ssim(dummy_data,torch.unsqueeze(gt_data[item],dim=0),data_range=0).item()
                    #recover_dis[iters]=torch.dist(dummy_data,torch.unsqueeze(gt_data[item],dim=0),2).item()
                    recover_dis[iters]= F.mse_loss(dummy_data,torch.unsqueeze(gt_data[item],dim=0)).item()
                    
                    history.append(tt(dummy_data[0].cpu()))

                    def closure():
                        optimizer.zero_grad()

                        pred = net(dummy_data) 
                        #dummy_onehot_label = F.softmax(dummy_label, dim=-1).long()

                        #dummy_loss = criterion(pred, dummy_onehot_label) # TODO: fix the gt_label to dummy_label in both code and slides.
                        #print (pred)
                        #print (label_pred)

                        dummy_loss = criterion(pred, label_pred)
                        dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)
                        #dummy_dy_dp = torch.autograd.grad(dummy_loss, dummy_data, create_graph=True)
                        #print (dummy_dy_dp[0].shape)  

                        grad_diff = 0
                        grad_count = 0
                        #count =0
                        for gx, gy in zip(dummy_dy_dx, original_dy_dx[item]): # TODO: fix the variablas here

                            #if iters==500 or iters== 1200:
                            #print (gx[0])
                            #    print ('hahaha')
                            #print (gy[0])
                            lasso = torch.norm(dummy_data,p=1)
                            ridge = torch.norm(dummy_data,p=2)
                            grad_diff += ((gx - gy) ** 2).sum() #+ 0.0*lasso +0.01*ridge 

                            #print (gx.shape)

                            grad_count += gx.nelement()


                            #if count == 9:
                            #    break
                            #count=count+1
                        # grad_diff = grad_diff / grad_count * 1000

                        #grad_diff += ((original_pred[item]-pred)**2).sum()




                        grad_diff.backward()
                        #print (count)

                        #print (dummy_dy_dx)
                        #print (original_dy_dx)


                        return grad_diff



                    optimizer.step(closure)
                    if iters % 5 == 0: 
                        current_loss = closure()
                        #if iters == 0: 
                        #print ("%.8f" % current_loss.item())
                        #print(iters, "%.8f" % current_loss.item())

                #     for bat in range(batch-1):
                #         history_batch.append(tt(dummy_data[bat].cpu()))

                plt.figure(figsize=(30, 20))
                for i in range(100):
                    plt.subplot(10, 10, i + 1)
                    plt.imshow(history[i * 5])
                    plt.title("iter=%d" % (i * 5))
                    plt.axis('off')
                #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
                
                #np.savetxt('lfw_ssim_%s'%iter_,percept_dis,fmt="%4f")
                #np.savetxt('lfw_mse_%s'%iter_,recover_dis,fmt="%4f")
                #plt.savefig("./attack_image/index_%s_iter_%s_label_%s"%(img_index,iter_,torch.argmax(dummy_label, dim=-1).item()))
                
                plt.clf()
            duration = time.process_time()-start
            #print ("Running time is %.4f." %(duration/10.0) )
            print ("Duration = ", duration)
            
            
            #if epoch>=1:
        #if i==1:
            #break
        #print (iter_)
        inputs,label = data

        inputs,label =  Variable(inputs),Variable(label) 

        optimizer_train.zero_grad()


        outputs_benign=net(inputs)
        #outputs_benign = F.softmax(outputs_benign, dim=-1)
        #print (outputs_benign[0])


        loss_benign =  criterion_train(outputs_benign,label)

        #print("loss computed")
        loss_benign.backward()
        #print("loss BP")
        optimizer_train.step()

        #if i%2000==0:
        print ("Loss Benign = ",loss_benign.item())
        #torch.save(net.state_dict(),'./LFW_net.pth')  

  
        print ('fininshed training')
        total = len(y_test)
        acc =0.0
        for ct in range(total):
            testing_data = tt(testing[ct][0].cpu())
            testing_data1 = transform(testing_data).to(device)
            testing_data2 = testing_data1.view(1, *testing_data1.size())
            y_pred = net(testing_data2)
            predicted = torch.argmax(y_pred)

            if predicted == y_test[ct]:
                acc=acc+1
        accuracy = acc / total
        print ("accuracy = ",accuracy)
        print ('fininshed testing')
        print("ssim_random2 = ", percept_dis)
        print("mse_random2 = ", recover_dis)

In [ ]:
plt.figure(figsize=(12, 8))
for i in range(60):
  plt.subplot(6, 10, i + 1)
  plt.imshow(history[i * 5])
  plt.title("iter=%d" % (i * 5))
  plt.axis('off')
print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())

In [ ]:
plt.imshow(history[295])

In [ ]:
plt.imshow(X_train[21], alpha =1)

# Noise Level 200

In [ ]:
# from pytorch_msssim import ssim

#dgc_rate_list = [20,30,40,50,70, 80,90]

#for epoch in range(1):
#for dgc_rate in dgc_rate_list:
gau_rate_list = [200]
for gau_rate in gau_rate_list:

    
    print('now starting',gau_rate)

    for iter_,data in enumerate(trainloader,1):
        
        if iter_ != 1:
            break
   
        ######### honest partipant #########
        img_index = 30   #use img_index
        dst_pil = tt(dst_tensor[img_index][0].cpu())   #use img_index

        gt_data = transform(dst_pil).to(device)
        gt_data = torch.unsqueeze(gt_data,0)

        gt_label = dst_tensor[img_index][1].long().to(device) #use img_index
        gt_label = gt_label.view(1, )
        gt_onehot_label = label_to_onehot(gt_label, num_classes=3)

        #plt.imshow(dst_pil)
        #plt.axis('off')
        plt.savefig("sample_data_COVID_Xray.png")



        batch =2  #
        for bat in range(batch-1):
            dst_pil = tt(dst_tensor[img_index+1+bat][0].cpu())   #use img_index
            tmp = torch.unsqueeze(transform(dst_pil).to(device),0)
            #print(tmp.shape)
            gt_data = torch.cat((gt_data,tmp),0)

            gt_label_tmp = dst_tensor[img_index+1+bat][1].long().to(device) #use img_index
            gt_label_tmp = gt_label_tmp.view(1, )
            gt_label = torch.cat((gt_label,gt_label_tmp),0)
            gt_onehot_label = torch.cat((gt_onehot_label,label_to_onehot(gt_label_tmp, num_classes=3)),0)

            plt.imshow(dst_pil)
            #plt.savefig("./original/index_%s_label_%s"%(bat+1,gt_label_tmp.item()))

            #plt.title("Ground truth image")
            #print("GT label is %d." % gt_label.item(), "\nOnehot label is %d." % torch.argmax(gt_onehot_label, dim=-1).item())


        gt_label = torch.reshape(gt_label,(-1,1))    
        #print (gt_data.shape)
        #print (gt_label.shape)
        #print (gt_onehot_label.shape)


        # compute original gradient 
        dy_dx = []
        original_dy_dx=[]
        original_pred = []
        for item in range(batch):
            gt_data_single = torch.unsqueeze(gt_data[item],0)
            out = net(gt_data_single)
            #y = criterion(out, gt_onehot_label[item])
            y = criterion(out, gt_label[item])
            dy_dx = torch.autograd.grad(y, net.parameters(),retain_graph=True)
            original_dy_dx_tmp = list((_.detach().clone() for _ in dy_dx))
            original_dy_dx.append(original_dy_dx_tmp)
            out_tmp = out.detach().clone()
            original_pred.append(out_tmp)
            
            
        #if gaussian noise or laplace
        m = torch.distributions.laplace.Laplace(torch.tensor([0.0]), torch.tensor([1/gau_rate]))
        for item in range(batch):
            for layer_idx in range(10):
                #original_dy_dx[item][layer_idx] =  original_dy_dx[item][layer_idx] + torch.empty(original_dy_dx[item][layer_idx].size()).normal_(mean=0,std=1/gau_rate).to(device)
                original_dy_dx[item][layer_idx] =  original_dy_dx[item][layer_idx] + torch.squeeze(m.sample(sample_shape=original_dy_dx[item][layer_idx].size()),dim=-1).to(device)
                #break
        ##if deep gradient compression
        #print (original_dy_dx[0][0][0])
        #for item in range(batch):
        #    for layer_idx in range(10):
        #        if layer_idx == 0:    
        #            flat_dy_dx = torch.flatten(original_dy_dx[item][layer_idx])
        #        else:
        #            flat_dy_dx = torch.cat((flat_dy_dx,torch.flatten(original_dy_dx[item][layer_idx])),0)
        #sorted_dy_dx = flat_dy_dx.abs().sort()
        #size = np.asarray(list(flat_dy_dx.shape))
        #thresh = sorted_dy_dx[0][int(size * dgc_rate/100.0)]
        #print (size)
        #print (int(size * dgc_rate/100.0))
        #print (thresh)
        #print (sorted_dy_dx[0][-1])
        #for item in range(batch):
        #    for layer_idx in range(10):
        #        shape_tmp = original_dy_dx[item][layer_idx].size()
        #        flat_dy_dx_prune = torch.flatten(original_dy_dx[item][layer_idx])
        #        size_tmp = np.asarray(list(flat_dy_dx_prune.shape))
        #        for m in range(int(size_tmp)):
        #            if flat_dy_dx_prune[m].abs()<=thresh:
        #                flat_dy_dx_prune[m] = 0
        #        original_dy_dx[item][layer_idx] = flat_dy_dx_prune.view(shape_tmp)
        #print (original_dy_dx[0][0][0])
              

        # generate dummy data and label
        import time

        #if iter_ % 10 ==0: 
        if iter_ == 1:
            
            #print ('epoch',epoch,'iter',iter_)
            for item in range(1):
                start = time.process_time()
                for rd in range(1):

                    torch.manual_seed(100*rd)

                    pat_1 = torch.rand([3,16,16])
                    pat_2 = torch.cat((pat_1,pat_1),dim=1)
                    pat_4 = torch.cat((pat_2,pat_2),dim=2)
                    dummy_data = torch.unsqueeze(pat_4,dim=0).to(device).requires_grad_(True)     

                    dummy_unsqueeze=torch.unsqueeze(gt_onehot_label[item],dim=0)

                    dummy_label = torch.randn(gt_onehot_label[item].size()).to(device).requires_grad_(True)
                    label_pred = torch.argmin(torch.sum(original_dy_dx[item][-2], dim=-1), dim=-1).detach().reshape((1,)).requires_grad_(False)
                    label_pred_onehot = label_to_onehot(label_pred, num_classes=3)

                    plt.imshow(tt(dummy_data[0].cpu()))
                    plt.title("Dummy data")
                    #plt.savefig("./random_seed/index_%s_rand_seed_%s_label_%s"%(item,rd,torch.argmax(dummy_label, dim=-1).item()))

                    plt.clf()
                    #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
                    #print("stolen label is %d." % label_pred.item())

                    #optimizer = torch.optim.LBFGS([dummy_data,dummy_label])
                    optimizer = torch.optim.LBFGS([dummy_data,])


                    history = []
                    history_batch = []
                    history_grad = []

                    percept_dis = np.zeros(100)
                    recover_dis = np.zeros(100)
                    for iters in range(100):

                        percept_dis[iters]=ssim(dummy_data,torch.unsqueeze(gt_data[item],dim=0),data_range=0).item()
                        recover_dis[iters]=torch.dist(dummy_data,torch.unsqueeze(gt_data[item],dim=0),2).item()

                        history.append(tt(dummy_data[0].cpu()))

                        def closure():
                            optimizer.zero_grad()

                            pred = net(dummy_data) 
                            dummy_onehot_label = F.softmax(dummy_label, dim=-1)
                            #dummy_loss = criterion(pred, dummy_onehot_label) # TODO: fix the gt_label to dummy_label in both code and slides.

                            #dummy_loss = criterion(pred, label_pred_onehot)
                            dummy_loss = criterion(pred, label_pred)


                            dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)
                            #dummy_dy_dp = torch.autograd.grad(dummy_loss, dummy_data, create_graph=True)
                            #print (dummy_dy_dp[0].shape)

                            grad_diff = 0
                            grad_count = 0
                            #count =0
                            for gx, gy in zip(dummy_dy_dx, original_dy_dx[item]): # TODO: fix the variablas here

                                #if iters==500 or iters== 1200:
                                #    print (gx[0])
                                #    print ('hahaha')
                                #    print (gy[0])
                                lasso = torch.norm(dummy_data,p=1)
                                ridge = torch.norm(dummy_data,p=2)
                                grad_diff += ((gx - gy) ** 2).sum() #+ 0.0*lasso +0.01*ridge


                                grad_count += gx.nelement()

                                #if count == 9:
                                #    break
                                #count=count+1
                            # grad_diff = grad_diff / grad_count * 1000
                            grad_diff.backward()
                            #print (count)

                            #print (dummy_dy_dx)
                            #print (original_dy_dx)


                            return grad_diff



                        optimizer.step(closure)
                        if iters % 5 == 0: 
                            current_loss = closure()
                            #if iters == 0: 
                            #print ("%.8f" % current_loss.item())
                            #print(iters, "%.8f" % current_loss.item())

                    #     for bat in range(batch-1):
                    #         history_batch.append(tt(dummy_data[bat].cpu()))

                    #plt.figure(figsize=(30, 20))
                    #for i in range(100):
                    #    plt.subplot(10, 10, i + 1)
                    #    plt.imshow(history[i * 5])
                    #    plt.title("iter=%d" % (i * 5))
                    #    plt.axis('off')
                    #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())

                    #np.savetxt('./attack_image/lfw_ssim_idx_%s_laplace_%s'%(item,gau_rate),percept_dis,fmt="%4f")
                    #np.savetxt('./attack_image/lfw_mse_idx_%s_laplace_%s'%(item,gau_rate),recover_dis,fmt="%4f")
                    #plt.savefig("./attack_image/lfw_index_%s_laplace_%s"%(item,gau_rate))

                    #plt.clf()
                    
                    pinp = np.argmin(recover_dis)
                    plt.imshow(history[pinp])
                    plt.axis('off')
                    
                    #plt.figure(figsize=(15, 10))
                    #for i in range(60):
                    #    plt.subplot(6, 10, i + 1)
                    #    plt.imshow(history[i * 14 ], cmap='gray')
                    #    plt.title("iter=%d" % (i*14))
                    #    plt.axis('off')
                    #plt.savefig("/content/sample_data"%(item,gau_rate))
                    
                #duration = time.clock()-start
                #print ("Running time is %.4f." %(duration/10.0) )
                #print (duration)
                
        
                
                
                
                
################################################### training when set to epoch

#         #if epoch>=1:
#         #if i==1:
#             #break
#         #print (iter_)
#         inputs,label = data

#         inputs,label =  Variable(inputs).to(device),Variable(label).to(device)

#         optimizer_train.zero_grad()


#         outputs_benign=net(inputs)
#         #outputs_benign = F.softmax(outputs_benign, dim=-1)
#         #print (outputs_benign[0])


#         loss_benign =  criterion_train(outputs_benign,label)

#         #print("loss computed")
#         loss_benign.backward()
#         #print("loss BP")
#         optimizer_train.step()
#         #sgd_update(net.parameters())

#         #if i%2000==0:
#         #print (loss_benign.item())
#         #torch.save(net.state_dict(),'./LFW_net.pth')  

#         #if  iter_%50==0:
#         #    print ('attack',iter_)
       
        
#         print ('fininshed training')
#         break
###############################   testing
    
#         total = len(y_test)
#         acc =0.0
#         for ct in range(total):
#             testing_data = tt(testing[ct][0].cpu())
#             testing_data1 = tp(testing_data).to(device)
#             testing_data2 = testing_data1.view(1, *testing_data1.size())
#             y_pred = net(testing_data2)
#             predicted = torch.argmax(y_pred)

#             if predicted == y_test[ct]:
#                 acc=acc+1
#         accuracy = acc / total
#         print (accuracy)
#         print ('fininshed testing')



In [ ]:
from pytorch_msssim import ssim


for epoch in range(1):

    for iter_,data in enumerate(trainloader,1):
   
        ######### honest partipant #########
        img_index = 30   #use img_index
        dst_pil = tt(dst_tensor[img_index][0].cpu())   #use img_index

        gt_data = transform(dst_pil).to(device)
        gt_data = torch.unsqueeze(gt_data,0)

        gt_label = dst_tensor[img_index][1].long().to(device) #use img_index
        gt_label = gt_label.view(1, )
        gt_onehot_label = label_to_onehot(gt_label, num_classes=3)

        plt.imshow(dst_pil)
        #plt.savefig("./original/index_%s_label_%s"%(img_index,gt_label.item()))



        batch =32  #
        for bat in range(batch-1):
            dst_pil = tt(dst_tensor[img_index+1+bat][0].cpu())   #use img_index
            tmp = torch.unsqueeze(transform(dst_pil).to(device),0)
            #print(tmp.shape)
            gt_data = torch.cat((gt_data,tmp),0)

            gt_label_tmp = dst_tensor[img_index+1+bat][1].long().to(device) #use img_index
            gt_label_tmp = gt_label_tmp.view(1, )
            gt_label = torch.cat((gt_label,gt_label_tmp),0)
            gt_onehot_label = torch.cat((gt_onehot_label,label_to_onehot(gt_label_tmp, num_classes=3)),0)

            plt.imshow(dst_pil)
            #plt.savefig("./original/index_%s_label_%s"%(bat+1,gt_label_tmp.item()))

            #plt.title("Ground truth image")
            #print("GT label is %d." % gt_label.item(), "\nOnehot label is %d." % torch.argmax(gt_onehot_label, dim=-1).item())


        gt_label = torch.reshape(gt_label,(-1,1))    
        print ("Image Shape = ",gt_data.shape)
        print ("Label shape = ",gt_label.shape)
        print ("Onehot Label shape = ",gt_onehot_label.shape)


        # compute original gradient 
        dy_dx = []
        original_dy_dx=[]
        original_pred = []
        for item in range(batch):
            gt_data_single = torch.unsqueeze(gt_data[item],0)
            out = net(gt_data_single)
            #y = criterion(out, gt_onehot_label[item])
            y = criterion(out, gt_label[item])
            dy_dx = torch.autograd.grad(y, net.parameters(),retain_graph=True)
            original_dy_dx_tmp = list((_.detach().clone() for _ in dy_dx))
            original_dy_dx.append(original_dy_dx_tmp)
            out_tmp = out.detach().clone()
            original_pred.append(out_tmp)

            #dy_dx.append(torch.autograd.grad(y, net.parameters()))





        # share the gradients with other clients
        #original_dy_dx = list((_.detach().clone() for _ in dy_dx))


        # generate dummy data and label
        import time


        for item in range(1):
            start = time.process_time()
            for rd in range(1):

                torch.manual_seed(100*rd)
        
                pat_1 = torch.rand([3,16,16])
                pat_2 = torch.cat((pat_1,pat_1),dim=1)
                pat_4 = torch.cat((pat_2,pat_2),dim=2)
                dummy_data = torch.unsqueeze(pat_4,dim=0).to(device).requires_grad_(True)     

                dummy_unsqueeze=torch.unsqueeze(gt_onehot_label[item],dim=0)

                dummy_label = torch.randn(dummy_unsqueeze.size()).to(device).requires_grad_(True)
                label_pred=torch.argmin(torch.sum(original_dy_dx[item][-2], dim=-1), 
                                        dim=-1).detach().reshape((1,)).requires_grad_(False)
                #print (original_dy_dx[item][-1].shape)
                #print (original_dy_dx[item][-1].argmin())

                #print (torch.sum(original_dy_dx[item][-2], dim=-1).argmin())

                plt.imshow(tt(dummy_data[0].cpu()))
                plt.title("Dummy data")
                #plt.savefig("./random_seed/index_%s_rand_seed_%s_label_%s"%(item,rd,torch.argmax(dummy_label, dim=-1).item()))

                plt.clf()
                print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
                print("stolen label is %d." % label_pred.item())


                #optimizer = torch.optim.LBFGS([dummy_data,dummy_label])
                optimizer = torch.optim.LBFGS([dummy_data,])
                #optimizer = torch.optim.AdamW([dummy_data,],lr=0.01)


                history = []
                history_batch = []
                history_grad = []
                
                percept_dis = np.zeros(500)
                recover_dis = np.zeros(500)
                for iters in range(500):
                    
                    percept_dis[iters]=ssim(dummy_data,torch.unsqueeze(gt_data[item],dim=0),data_range=0).item()
                    #recover_dis[iters]=torch.dist(dummy_data,torch.unsqueeze(gt_data[item],dim=0),2).item()
                    recover_dis[iters]= F.mse_loss(dummy_data,torch.unsqueeze(gt_data[item],dim=0)).item()
                    
                    history.append(tt(dummy_data[0].cpu()))

                    def closure():
                        optimizer.zero_grad()

                        pred = net(dummy_data) 
                        #dummy_onehot_label = F.softmax(dummy_label, dim=-1).long()

                        #dummy_loss = criterion(pred, dummy_onehot_label) # TODO: fix the gt_label to dummy_label in both code and slides.
                        #print (pred)
                        #print (label_pred)

                        dummy_loss = criterion(pred, label_pred)
                        dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)
                        #dummy_dy_dp = torch.autograd.grad(dummy_loss, dummy_data, create_graph=True)
                        #print (dummy_dy_dp[0].shape)  

                        grad_diff = 0
                        grad_count = 0
                        #count =0
                        for gx, gy in zip(dummy_dy_dx, original_dy_dx[item]): # TODO: fix the variablas here

                            #if iters==500 or iters== 1200:
                            #print (gx[0])
                            #    print ('hahaha')
                            #print (gy[0])
                            lasso = torch.norm(dummy_data,p=1)
                            ridge = torch.norm(dummy_data,p=2)
                            grad_diff += ((gx - gy) ** 2).sum() #+ 0.0*lasso +0.01*ridge 

                            #print (gx.shape)

                            grad_count += gx.nelement()


                            #if count == 9:
                            #    break
                            #count=count+1
                        # grad_diff = grad_diff / grad_count * 1000

                        #grad_diff += ((original_pred[item]-pred)**2).sum()




                        grad_diff.backward()
                        #print (count)

                        #print (dummy_dy_dx)
                        #print (original_dy_dx)


                        return grad_diff



                    optimizer.step(closure)
                    if iters % 5 == 0: 
                        current_loss = closure()
                        #if iters == 0: 
                        #print ("%.8f" % current_loss.item())
                        #print(iters, "%.8f" % current_loss.item())

                #     for bat in range(batch-1):
                #         history_batch.append(tt(dummy_data[bat].cpu()))

                plt.figure(figsize=(30, 20))
                for i in range(100):
                    plt.subplot(10, 10, i + 1)
                    plt.imshow(history[i * 5])
                    plt.title("iter=%d" % (i * 5))
                    plt.axis('off')
                #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
                
                #np.savetxt('lfw_ssim_%s'%iter_,percept_dis,fmt="%4f")
                #np.savetxt('lfw_mse_%s'%iter_,recover_dis,fmt="%4f")
                #plt.savefig("./attack_image/index_%s_iter_%s_label_%s"%(img_index,iter_,torch.argmax(dummy_label, dim=-1).item()))
                
                plt.clf()
            duration = time.process_time()-start
            #print ("Running time is %.4f." %(duration/10.0) )
            print ("Duration = ", duration)
            
            
            #if epoch>=1:
        #if i==1:
            #break
        #print (iter_)
        inputs,label = data

        inputs,label =  Variable(inputs),Variable(label) 

        optimizer_train.zero_grad()


        outputs_benign=net(inputs)
        #outputs_benign = F.softmax(outputs_benign, dim=-1)
        #print (outputs_benign[0])


        loss_benign =  criterion_train(outputs_benign,label)

        #print("loss computed")
        loss_benign.backward()
        #print("loss BP")
        optimizer_train.step()

        #if i%2000==0:
        print ("Loss Benign = ",loss_benign.item())
        #torch.save(net.state_dict(),'./LFW_net.pth')  

  
        print ('fininshed training')
        total = len(y_test)
        acc =0.0
        for ct in range(total):
            testing_data = tt(testing[ct][0].cpu())
            testing_data1 = transform(testing_data).to(device)
            testing_data2 = testing_data1.view(1, *testing_data1.size())
            y_pred = net(testing_data2)
            predicted = torch.argmax(y_pred)

            if predicted == y_test[ct]:
                acc=acc+1
        accuracy = acc / total
        print ("accuracy = ",accuracy)
        print ('fininshed testing')
        print("ssim_random2 = ", percept_dis)
        print("mse_random2 = ", recover_dis)


        

In [ ]:
plt.figure(figsize=(12, 8))
for i in range(60):
  plt.subplot(6, 10, i + 1)
  plt.imshow(history[i * 5])
  plt.title("iter=%d" % (i * 5))
  plt.axis('off')
print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())

# Noise Level 300

In [ ]:
from pytorch_msssim import ssim

#dgc_rate_list = [20,30,40,50,70, 80,90]

#for epoch in range(1):
#for dgc_rate in dgc_rate_list:
gau_rate_list = [300]
for gau_rate in gau_rate_list:

    
    print('now starting',gau_rate)

    for iter_,data in enumerate(trainloader,1):
        
        if iter_ != 1:
            break
   
        ######### honest partipant #########
        img_index = 30   #use img_index
        dst_pil = tt(dst_tensor[img_index][0].cpu())   #use img_index

        gt_data = transform(dst_pil).to(device)
        gt_data = torch.unsqueeze(gt_data,0)

        gt_label = dst_tensor[img_index][1].long().to(device) #use img_index
        gt_label = gt_label.view(1, )
        gt_onehot_label = label_to_onehot(gt_label, num_classes=3)

        #plt.imshow(dst_pil)
        #plt.axis('off')
        #plt.savefig("/content/sample_data")



        batch =2  #
        for bat in range(batch-1):
            dst_pil = tt(dst_tensor[img_index+1+bat][0].cpu())   #use img_index
            tmp = torch.unsqueeze(transform(dst_pil).to(device),0)
            #print(tmp.shape)
            gt_data = torch.cat((gt_data,tmp),0)

            gt_label_tmp = dst_tensor[img_index+1+bat][1].long().to(device) #use img_index
            gt_label_tmp = gt_label_tmp.view(1, )
            gt_label = torch.cat((gt_label,gt_label_tmp),0)
            gt_onehot_label = torch.cat((gt_onehot_label,label_to_onehot(gt_label_tmp, num_classes=3)),0)

            plt.imshow(dst_pil)
            #plt.savefig("./original/index_%s_label_%s"%(bat+1,gt_label_tmp.item()))

            #plt.title("Ground truth image")
            #print("GT label is %d." % gt_label.item(), "\nOnehot label is %d." % torch.argmax(gt_onehot_label, dim=-1).item())


        gt_label = torch.reshape(gt_label,(-1,1))    
        #print (gt_data.shape)
        #print (gt_label.shape)
        #print (gt_onehot_label.shape)


        # compute original gradient 
        dy_dx = []
        original_dy_dx=[]
        original_pred = []
        for item in range(batch):
            gt_data_single = torch.unsqueeze(gt_data[item],0)
            out = net(gt_data_single)
            #y = criterion(out, gt_onehot_label[item])
            y = criterion(out, gt_label[item])
            dy_dx = torch.autograd.grad(y, net.parameters(),retain_graph=True)
            original_dy_dx_tmp = list((_.detach().clone() for _ in dy_dx))
            original_dy_dx.append(original_dy_dx_tmp)
            out_tmp = out.detach().clone()
            original_pred.append(out_tmp)
            
            
        #if gaussian noise or laplace
        m = torch.distributions.laplace.Laplace(torch.tensor([0.0]), torch.tensor([1/gau_rate]))
        for item in range(batch):
            for layer_idx in range(10):
                #original_dy_dx[item][layer_idx] =  original_dy_dx[item][layer_idx] + torch.empty(original_dy_dx[item][layer_idx].size()).normal_(mean=0,std=1/gau_rate).to(device)
                original_dy_dx[item][layer_idx] =  original_dy_dx[item][layer_idx] + torch.squeeze(m.sample(sample_shape=original_dy_dx[item][layer_idx].size()),dim=-1).to(device)
                #break
        ##if deep gradient compression
        #print (original_dy_dx[0][0][0])
        #for item in range(batch):
        #    for layer_idx in range(10):
        #        if layer_idx == 0:    
        #            flat_dy_dx = torch.flatten(original_dy_dx[item][layer_idx])
        #        else:
        #            flat_dy_dx = torch.cat((flat_dy_dx,torch.flatten(original_dy_dx[item][layer_idx])),0)
        #sorted_dy_dx = flat_dy_dx.abs().sort()
        #size = np.asarray(list(flat_dy_dx.shape))
        #thresh = sorted_dy_dx[0][int(size * dgc_rate/100.0)]
        #print (size)
        #print (int(size * dgc_rate/100.0))
        #print (thresh)
        #print (sorted_dy_dx[0][-1])
        #for item in range(batch):
        #    for layer_idx in range(10):
        #        shape_tmp = original_dy_dx[item][layer_idx].size()
        #        flat_dy_dx_prune = torch.flatten(original_dy_dx[item][layer_idx])
        #        size_tmp = np.asarray(list(flat_dy_dx_prune.shape))
        #        for m in range(int(size_tmp)):
        #            if flat_dy_dx_prune[m].abs()<=thresh:
        #                flat_dy_dx_prune[m] = 0
        #        original_dy_dx[item][layer_idx] = flat_dy_dx_prune.view(shape_tmp)
        #print (original_dy_dx[0][0][0])
              

        # generate dummy data and label
        import time

        #if iter_ % 10 ==0: 
        if iter_ == 1:
            
            #print ('epoch',epoch,'iter',iter_)
            for item in range(1):
                start = time.process_time()
                for rd in range(1):

                    torch.manual_seed(100*rd)

                    pat_1 = torch.rand([3,16,16])
                    pat_2 = torch.cat((pat_1,pat_1),dim=1)
                    pat_4 = torch.cat((pat_2,pat_2),dim=2)
                    dummy_data = torch.unsqueeze(pat_4,dim=0).to(device).requires_grad_(True)     

                    dummy_unsqueeze=torch.unsqueeze(gt_onehot_label[item],dim=0)

                    dummy_label = torch.randn(gt_onehot_label[item].size()).to(device).requires_grad_(True)
                    label_pred = torch.argmin(torch.sum(original_dy_dx[item][-2], dim=-1), dim=-1).detach().reshape((1,)).requires_grad_(False)
                    label_pred_onehot = label_to_onehot(label_pred, num_classes=3)

                    plt.imshow(tt(dummy_data[0].cpu()))
                    plt.title("Dummy data")
                    #plt.savefig("./random_seed/index_%s_rand_seed_%s_label_%s"%(item,rd,torch.argmax(dummy_label, dim=-1).item()))

                    plt.clf()
                    #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
                    #print("stolen label is %d." % label_pred.item())

                    #optimizer = torch.optim.LBFGS([dummy_data,dummy_label])
                    optimizer = torch.optim.LBFGS([dummy_data,])


                    history = []
                    history_batch = []
                    history_grad = []

                    percept_dis = np.zeros(100)
                    recover_dis = np.zeros(100)
                    for iters in range(100):

                        percept_dis[iters]=ssim(dummy_data,torch.unsqueeze(gt_data[item],dim=0),data_range=0).item()
                        recover_dis[iters]=torch.dist(dummy_data,torch.unsqueeze(gt_data[item],dim=0),2).item()

                        history.append(tt(dummy_data[0].cpu()))

                        def closure():
                            optimizer.zero_grad()

                            pred = net(dummy_data) 
                            dummy_onehot_label = F.softmax(dummy_label, dim=-1)
                            #dummy_loss = criterion(pred, dummy_onehot_label) # TODO: fix the gt_label to dummy_label in both code and slides.

                            #dummy_loss = criterion(pred, label_pred_onehot)
                            dummy_loss = criterion(pred, label_pred)


                            dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)
                            #dummy_dy_dp = torch.autograd.grad(dummy_loss, dummy_data, create_graph=True)
                            #print (dummy_dy_dp[0].shape)

                            grad_diff = 0
                            grad_count = 0
                            #count =0
                            for gx, gy in zip(dummy_dy_dx, original_dy_dx[item]): # TODO: fix the variablas here

                                #if iters==500 or iters== 1200:
                                #    print (gx[0])
                                #    print ('hahaha')
                                #    print (gy[0])
                                lasso = torch.norm(dummy_data,p=1)
                                ridge = torch.norm(dummy_data,p=2)
                                grad_diff += ((gx - gy) ** 2).sum() #+ 0.0*lasso +0.01*ridge


                                grad_count += gx.nelement()

                                #if count == 9:
                                #    break
                                #count=count+1
                            # grad_diff = grad_diff / grad_count * 1000
                            grad_diff.backward()
                            #print (count)

                            #print (dummy_dy_dx)
                            #print (original_dy_dx)


                            return grad_diff



                        optimizer.step(closure)
                        if iters % 5 == 0: 
                            current_loss = closure()
                            #if iters == 0: 
                            #print ("%.8f" % current_loss.item())
                            #print(iters, "%.8f" % current_loss.item())

                    #     for bat in range(batch-1):
                    #         history_batch.append(tt(dummy_data[bat].cpu()))

                    #plt.figure(figsize=(30, 20))
                    #for i in range(100):
                    #    plt.subplot(10, 10, i + 1)
                    #    plt.imshow(history[i * 5])
                    #    plt.title("iter=%d" % (i * 5))
                    #    plt.axis('off')
                    #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())

                    #np.savetxt('./attack_image/lfw_ssim_idx_%s_laplace_%s'%(item,gau_rate),percept_dis,fmt="%4f")
                    #np.savetxt('./attack_image/lfw_mse_idx_%s_laplace_%s'%(item,gau_rate),recover_dis,fmt="%4f")
                    #plt.savefig("./attack_image/lfw_index_%s_laplace_%s"%(item,gau_rate))

                    #plt.clf()
                    
                    pinp = np.argmin(recover_dis)
                    plt.imshow(history[pinp])
                    plt.axis('off')
                    
                    #plt.figure(figsize=(15, 10))
                    #for i in range(60):
                    #    plt.subplot(6, 10, i + 1)
                    #    plt.imshow(history[i * 14 ], cmap='gray')
                    #    plt.title("iter=%d" % (i*14))
                    #    plt.axis('off')
                    #plt.savefig("/content/sample_data"%(item,gau_rate))
                    
                #duration = time.clock()-start
                #print ("Running time is %.4f." %(duration/10.0) )
                #print (duration)
                
        
                
                
                
                
################################################### training when set to epoch

#         #if epoch>=1:
#         #if i==1:
#             #break
#         #print (iter_)
#         inputs,label = data

#         inputs,label =  Variable(inputs).to(device),Variable(label).to(device)

#         optimizer_train.zero_grad()


#         outputs_benign=net(inputs)
#         #outputs_benign = F.softmax(outputs_benign, dim=-1)
#         #print (outputs_benign[0])


#         loss_benign =  criterion_train(outputs_benign,label)

#         #print("loss computed")
#         loss_benign.backward()
#         #print("loss BP")
#         optimizer_train.step()
#         #sgd_update(net.parameters())

#         #if i%2000==0:
#         #print (loss_benign.item())
#         #torch.save(net.state_dict(),'./LFW_net.pth')  

#         #if  iter_%50==0:
#         #    print ('attack',iter_)
       
        
#         print ('fininshed training')
#         break
###############################   testing
    
#         total = len(y_test)
#         acc =0.0
#         for ct in range(total):
#             testing_data = tt(testing[ct][0].cpu())
#             testing_data1 = tp(testing_data).to(device)
#             testing_data2 = testing_data1.view(1, *testing_data1.size())
#             y_pred = net(testing_data2)
#             predicted = torch.argmax(y_pred)

#             if predicted == y_test[ct]:
#                 acc=acc+1
#         accuracy = acc / total
#         print (accuracy)
#         print ('fininshed testing')



In [ ]:
from pytorch_msssim import ssim


for epoch in range(1):

    for iter_,data in enumerate(trainloader,1):
   
        ######### honest partipant #########
        img_index = 30   #use img_index
        dst_pil = tt(dst_tensor[img_index][0].cpu())   #use img_index

        gt_data = transform(dst_pil).to(device)
        gt_data = torch.unsqueeze(gt_data,0)

        gt_label = dst_tensor[img_index][1].long().to(device) #use img_index
        gt_label = gt_label.view(1, )
        gt_onehot_label = label_to_onehot(gt_label, num_classes=3)

        plt.imshow(dst_pil)
        #plt.savefig("./original/index_%s_label_%s"%(img_index,gt_label.item()))



        batch =32  #
        for bat in range(batch-1):
            dst_pil = tt(dst_tensor[img_index+1+bat][0].cpu())   #use img_index
            tmp = torch.unsqueeze(transform(dst_pil).to(device),0)
            #print(tmp.shape)
            gt_data = torch.cat((gt_data,tmp),0)

            gt_label_tmp = dst_tensor[img_index+1+bat][1].long().to(device) #use img_index
            gt_label_tmp = gt_label_tmp.view(1, )
            gt_label = torch.cat((gt_label,gt_label_tmp),0)
            gt_onehot_label = torch.cat((gt_onehot_label,label_to_onehot(gt_label_tmp, num_classes=3)),0)

            plt.imshow(dst_pil)
            #plt.savefig("./original/index_%s_label_%s"%(bat+1,gt_label_tmp.item()))

            #plt.title("Ground truth image")
            #print("GT label is %d." % gt_label.item(), "\nOnehot label is %d." % torch.argmax(gt_onehot_label, dim=-1).item())


        gt_label = torch.reshape(gt_label,(-1,1))    
        print ("Image Shape = ",gt_data.shape)
        print ("Label shape = ",gt_label.shape)
        print ("Onehot Label shape = ",gt_onehot_label.shape)


        # compute original gradient 
        dy_dx = []
        original_dy_dx=[]
        original_pred = []
        for item in range(batch):
            gt_data_single = torch.unsqueeze(gt_data[item],0)
            out = net(gt_data_single)
            #y = criterion(out, gt_onehot_label[item])
            y = criterion(out, gt_label[item])
            dy_dx = torch.autograd.grad(y, net.parameters(),retain_graph=True)
            original_dy_dx_tmp = list((_.detach().clone() for _ in dy_dx))
            original_dy_dx.append(original_dy_dx_tmp)
            out_tmp = out.detach().clone()
            original_pred.append(out_tmp)

            #dy_dx.append(torch.autograd.grad(y, net.parameters()))





        # share the gradients with other clients
        #original_dy_dx = list((_.detach().clone() for _ in dy_dx))


        # generate dummy data and label
        import time


        for item in range(1):
            start = time.process_time()
            for rd in range(1):

                torch.manual_seed(100*rd)
        
                pat_1 = torch.rand([3,16,16])
                pat_2 = torch.cat((pat_1,pat_1),dim=1)
                pat_4 = torch.cat((pat_2,pat_2),dim=2)
                dummy_data = torch.unsqueeze(pat_4,dim=0).to(device).requires_grad_(True)     

                dummy_unsqueeze=torch.unsqueeze(gt_onehot_label[item],dim=0)

                dummy_label = torch.randn(dummy_unsqueeze.size()).to(device).requires_grad_(True)
                label_pred=torch.argmin(torch.sum(original_dy_dx[item][-2], dim=-1), 
                                        dim=-1).detach().reshape((1,)).requires_grad_(False)
                #print (original_dy_dx[item][-1].shape)
                #print (original_dy_dx[item][-1].argmin())

                #print (torch.sum(original_dy_dx[item][-2], dim=-1).argmin())

                plt.imshow(tt(dummy_data[0].cpu()))
                plt.title("Dummy data")
                #plt.savefig("./random_seed/index_%s_rand_seed_%s_label_%s"%(item,rd,torch.argmax(dummy_label, dim=-1).item()))

                plt.clf()
                print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
                print("stolen label is %d." % label_pred.item())


                #optimizer = torch.optim.LBFGS([dummy_data,dummy_label])
                optimizer = torch.optim.LBFGS([dummy_data,])
                #optimizer = torch.optim.AdamW([dummy_data,],lr=0.01)


                history = []
                history_batch = []
                history_grad = []
                
                percept_dis = np.zeros(500)
                recover_dis = np.zeros(500)
                for iters in range(500):
                    
                    percept_dis[iters]=ssim(dummy_data,torch.unsqueeze(gt_data[item],dim=0),data_range=0).item()
                    #recover_dis[iters]=torch.dist(dummy_data,torch.unsqueeze(gt_data[item],dim=0),2).item()
                    recover_dis[iters]= F.mse_loss(dummy_data,torch.unsqueeze(gt_data[item],dim=0)).item()
                    
                    history.append(tt(dummy_data[0].cpu()))

                    def closure():
                        optimizer.zero_grad()

                        pred = net(dummy_data) 
                        #dummy_onehot_label = F.softmax(dummy_label, dim=-1).long()

                        #dummy_loss = criterion(pred, dummy_onehot_label) # TODO: fix the gt_label to dummy_label in both code and slides.
                        #print (pred)
                        #print (label_pred)

                        dummy_loss = criterion(pred, label_pred)
                        dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)
                        #dummy_dy_dp = torch.autograd.grad(dummy_loss, dummy_data, create_graph=True)
                        #print (dummy_dy_dp[0].shape)  

                        grad_diff = 0
                        grad_count = 0
                        #count =0
                        for gx, gy in zip(dummy_dy_dx, original_dy_dx[item]): # TODO: fix the variablas here

                            #if iters==500 or iters== 1200:
                            #print (gx[0])
                            #    print ('hahaha')
                            #print (gy[0])
                            lasso = torch.norm(dummy_data,p=1)
                            ridge = torch.norm(dummy_data,p=2)
                            grad_diff += ((gx - gy) ** 2).sum() #+ 0.0*lasso +0.01*ridge 

                            #print (gx.shape)

                            grad_count += gx.nelement()


                            #if count == 9:
                            #    break
                            #count=count+1
                        # grad_diff = grad_diff / grad_count * 1000

                        #grad_diff += ((original_pred[item]-pred)**2).sum()




                        grad_diff.backward()
                        #print (count)

                        #print (dummy_dy_dx)
                        #print (original_dy_dx)


                        return grad_diff



                    optimizer.step(closure)
                    if iters % 5 == 0: 
                        current_loss = closure()
                        #if iters == 0: 
                        #print ("%.8f" % current_loss.item())
                        #print(iters, "%.8f" % current_loss.item())

                #     for bat in range(batch-1):
                #         history_batch.append(tt(dummy_data[bat].cpu()))

                plt.figure(figsize=(30, 20))
                for i in range(100):
                    plt.subplot(10, 10, i + 1)
                    plt.imshow(history[i * 5])
                    plt.title("iter=%d" % (i * 5))
                    plt.axis('off')
                #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
                
                #np.savetxt('lfw_ssim_%s'%iter_,percept_dis,fmt="%4f")
                #np.savetxt('lfw_mse_%s'%iter_,recover_dis,fmt="%4f")
                #plt.savefig("./attack_image/index_%s_iter_%s_label_%s"%(img_index,iter_,torch.argmax(dummy_label, dim=-1).item()))
                
                plt.clf()
            duration = time.process_time()-start
            #print ("Running time is %.4f." %(duration/10.0) )
            print ("Duration = ", duration)
            
            
            #if epoch>=1:
        #if i==1:
            #break
        #print (iter_)
        inputs,label = data

        inputs,label =  Variable(inputs),Variable(label) 

        optimizer_train.zero_grad()


        outputs_benign=net(inputs)
        #outputs_benign = F.softmax(outputs_benign, dim=-1)
        #print (outputs_benign[0])


        loss_benign =  criterion_train(outputs_benign,label)

        #print("loss computed")
        loss_benign.backward()
        #print("loss BP")
        optimizer_train.step()

        #if i%2000==0:
        print ("Loss Benign = ",loss_benign.item())
        #torch.save(net.state_dict(),'./LFW_net.pth')  

  
        print ('fininshed training')
        total = len(y_test)
        acc =0.0
        for ct in range(total):
            testing_data = tt(testing[ct][0].cpu())
            testing_data1 = transform(testing_data).to(device)
            testing_data2 = testing_data1.view(1, *testing_data1.size())
            y_pred = net(testing_data2)
            predicted = torch.argmax(y_pred)

            if predicted == y_test[ct]:
                acc=acc+1
        accuracy = acc / total
        print ("accuracy = ",accuracy)
        print ('fininshed testing')
        print("ssim_random2 = ", percept_dis)
        print("mse_random2 = ", recover_dis)


        

In [ ]:
plt.figure(figsize=(12, 8))
for i in range(60):
  plt.subplot(6, 10, i + 1)
  plt.imshow(history[i * 5])
  plt.title("iter=%d" % (i * 5))
  plt.axis('off')
print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())

# Noise level 400

In [ ]:
from pytorch_msssim import ssim

#dgc_rate_list = [20,30,40,50,70, 80,90]

#for epoch in range(1):
#for dgc_rate in dgc_rate_list:
gau_rate_list = [400]
for gau_rate in gau_rate_list:

    
    print('now starting',gau_rate)

    for iter_,data in enumerate(trainloader,1):
        
        if iter_ != 1:
            break
   
        ######### honest partipant #########
        img_index = 30   #use img_index
        dst_pil = tt(dst_tensor[img_index][0].cpu())   #use img_index

        gt_data = transform(dst_pil).to(device)
        gt_data = torch.unsqueeze(gt_data,0)

        gt_label = dst_tensor[img_index][1].long().to(device) #use img_index
        gt_label = gt_label.view(1, )
        gt_onehot_label = label_to_onehot(gt_label, num_classes=3)

        #plt.imshow(dst_pil)
        #plt.axis('off')
        #plt.savefig("sample_data_COVID_Xray")



        batch =2  #
        for bat in range(batch-1):
            dst_pil = tt(dst_tensor[img_index+1+bat][0].cpu())   #use img_index
            tmp = torch.unsqueeze(transform(dst_pil).to(device),0)
            #print(tmp.shape)
            gt_data = torch.cat((gt_data,tmp),0)

            gt_label_tmp = dst_tensor[img_index+1+bat][1].long().to(device) #use img_index
            gt_label_tmp = gt_label_tmp.view(1, )
            gt_label = torch.cat((gt_label,gt_label_tmp),0)
            gt_onehot_label = torch.cat((gt_onehot_label,label_to_onehot(gt_label_tmp, num_classes=3)),0)

            plt.imshow(dst_pil)
            #plt.savefig("./original/index_%s_label_%s"%(bat+1,gt_label_tmp.item()))

            #plt.title("Ground truth image")
            #print("GT label is %d." % gt_label.item(), "\nOnehot label is %d." % torch.argmax(gt_onehot_label, dim=-1).item())


        gt_label = torch.reshape(gt_label,(-1,1))    
        #print (gt_data.shape)
        #print (gt_label.shape)
        #print (gt_onehot_label.shape)


        # compute original gradient 
        dy_dx = []
        original_dy_dx=[]
        original_pred = []
        for item in range(batch):
            gt_data_single = torch.unsqueeze(gt_data[item],0)
            out = net(gt_data_single)
            #y = criterion(out, gt_onehot_label[item])
            y = criterion(out, gt_label[item])
            dy_dx = torch.autograd.grad(y, net.parameters(),retain_graph=True)
            original_dy_dx_tmp = list((_.detach().clone() for _ in dy_dx))
            original_dy_dx.append(original_dy_dx_tmp)
            out_tmp = out.detach().clone()
            original_pred.append(out_tmp)
            
            
        #if gaussian noise or laplace
        m = torch.distributions.laplace.Laplace(torch.tensor([0.0]), torch.tensor([1/gau_rate]))
        for item in range(batch):
            for layer_idx in range(10):
                #original_dy_dx[item][layer_idx] =  original_dy_dx[item][layer_idx] + torch.empty(original_dy_dx[item][layer_idx].size()).normal_(mean=0,std=1/gau_rate).to(device)
                original_dy_dx[item][layer_idx] =  original_dy_dx[item][layer_idx] + torch.squeeze(m.sample(sample_shape=original_dy_dx[item][layer_idx].size()),dim=-1).to(device)
                #break
        ##if deep gradient compression
        #print (original_dy_dx[0][0][0])
        #for item in range(batch):
        #    for layer_idx in range(10):
        #        if layer_idx == 0:    
        #            flat_dy_dx = torch.flatten(original_dy_dx[item][layer_idx])
        #        else:
        #            flat_dy_dx = torch.cat((flat_dy_dx,torch.flatten(original_dy_dx[item][layer_idx])),0)
        #sorted_dy_dx = flat_dy_dx.abs().sort()
        #size = np.asarray(list(flat_dy_dx.shape))
        #thresh = sorted_dy_dx[0][int(size * dgc_rate/100.0)]
        #print (size)
        #print (int(size * dgc_rate/100.0))
        #print (thresh)
        #print (sorted_dy_dx[0][-1])
        #for item in range(batch):
        #    for layer_idx in range(10):
        #        shape_tmp = original_dy_dx[item][layer_idx].size()
        #        flat_dy_dx_prune = torch.flatten(original_dy_dx[item][layer_idx])
        #        size_tmp = np.asarray(list(flat_dy_dx_prune.shape))
        #        for m in range(int(size_tmp)):
        #            if flat_dy_dx_prune[m].abs()<=thresh:
        #                flat_dy_dx_prune[m] = 0
        #        original_dy_dx[item][layer_idx] = flat_dy_dx_prune.view(shape_tmp)
        #print (original_dy_dx[0][0][0])
              

        # generate dummy data and label
        import time

        #if iter_ % 10 ==0: 
        if iter_ == 1:
            
            #print ('epoch',epoch,'iter',iter_)
            for item in range(1):
                start = time.process_time()
                for rd in range(1):

                    torch.manual_seed(100*rd)

                    pat_1 = torch.rand([3,16,16])
                    pat_2 = torch.cat((pat_1,pat_1),dim=1)
                    pat_4 = torch.cat((pat_2,pat_2),dim=2)
                    dummy_data = torch.unsqueeze(pat_4,dim=0).to(device).requires_grad_(True)     

                    dummy_unsqueeze=torch.unsqueeze(gt_onehot_label[item],dim=0)

                    dummy_label = torch.randn(gt_onehot_label[item].size()).to(device).requires_grad_(True)
                    label_pred = torch.argmin(torch.sum(original_dy_dx[item][-2], dim=-1), dim=-1).detach().reshape((1,)).requires_grad_(False)
                    label_pred_onehot = label_to_onehot(label_pred, num_classes=3)

                    plt.imshow(tt(dummy_data[0].cpu()))
                    plt.title("Dummy data")
                    #plt.savefig("./random_seed/index_%s_rand_seed_%s_label_%s"%(item,rd,torch.argmax(dummy_label, dim=-1).item()))

                    plt.clf()
                    #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
                    #print("stolen label is %d." % label_pred.item())

                    #optimizer = torch.optim.LBFGS([dummy_data,dummy_label])
                    optimizer = torch.optim.LBFGS([dummy_data,])


                    history = []
                    history_batch = []
                    history_grad = []

                    percept_dis = np.zeros(100)
                    recover_dis = np.zeros(100)
                    for iters in range(100):

                        percept_dis[iters]=ssim(dummy_data,torch.unsqueeze(gt_data[item],dim=0),data_range=0).item()
                        recover_dis[iters]=torch.dist(dummy_data,torch.unsqueeze(gt_data[item],dim=0),2).item()

                        history.append(tt(dummy_data[0].cpu()))

                        def closure():
                            optimizer.zero_grad()

                            pred = net(dummy_data) 
                            dummy_onehot_label = F.softmax(dummy_label, dim=-1)
                            #dummy_loss = criterion(pred, dummy_onehot_label) # TODO: fix the gt_label to dummy_label in both code and slides.

                            #dummy_loss = criterion(pred, label_pred_onehot)
                            dummy_loss = criterion(pred, label_pred)


                            dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)
                            #dummy_dy_dp = torch.autograd.grad(dummy_loss, dummy_data, create_graph=True)
                            #print (dummy_dy_dp[0].shape)

                            grad_diff = 0
                            grad_count = 0
                            #count =0
                            for gx, gy in zip(dummy_dy_dx, original_dy_dx[item]): # TODO: fix the variablas here

                                #if iters==500 or iters== 1200:
                                #    print (gx[0])
                                #    print ('hahaha')
                                #    print (gy[0])
                                lasso = torch.norm(dummy_data,p=1)
                                ridge = torch.norm(dummy_data,p=2)
                                grad_diff += ((gx - gy) ** 2).sum() #+ 0.0*lasso +0.01*ridge


                                grad_count += gx.nelement()

                                #if count == 9:
                                #    break
                                #count=count+1
                            # grad_diff = grad_diff / grad_count * 1000
                            grad_diff.backward()
                            #print (count)

                            #print (dummy_dy_dx)
                            #print (original_dy_dx)


                            return grad_diff



                        optimizer.step(closure)
                        if iters % 5 == 0: 
                            current_loss = closure()
                            #if iters == 0: 
                            #print ("%.8f" % current_loss.item())
                            #print(iters, "%.8f" % current_loss.item())

                    #     for bat in range(batch-1):
                    #         history_batch.append(tt(dummy_data[bat].cpu()))

                    #plt.figure(figsize=(30, 20))
                    #for i in range(100):
                    #    plt.subplot(10, 10, i + 1)
                    #    plt.imshow(history[i * 5])
                    #    plt.title("iter=%d" % (i * 5))
                    #    plt.axis('off')
                    #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())

                    #np.savetxt('./attack_image/lfw_ssim_idx_%s_laplace_%s'%(item,gau_rate),percept_dis,fmt="%4f")
                    #np.savetxt('./attack_image/lfw_mse_idx_%s_laplace_%s'%(item,gau_rate),recover_dis,fmt="%4f")
                    #plt.savefig("./attack_image/lfw_index_%s_laplace_%s"%(item,gau_rate))

                    #plt.clf()
                    
                    pinp = np.argmin(recover_dis)
                    plt.imshow(history[pinp])
                    plt.axis('off')
                    
                    #plt.figure(figsize=(15, 10))
                    #for i in range(60):
                    #    plt.subplot(6, 10, i + 1)
                    #    plt.imshow(history[i * 14 ], cmap='gray')
                    #    plt.title("iter=%d" % (i*14))
                    #    plt.axis('off')
                    #plt.savefig("/content/sample_data"%(item,gau_rate))
                    
                #duration = time.clock()-start
                #print ("Running time is %.4f." %(duration/10.0) )
                #print (duration)
                
        
                
                
                
                
################################################### training when set to epoch

#         #if epoch>=1:
#         #if i==1:
#             #break
#         #print (iter_)
#         inputs,label = data

#         inputs,label =  Variable(inputs).to(device),Variable(label).to(device)

#         optimizer_train.zero_grad()


#         outputs_benign=net(inputs)
#         #outputs_benign = F.softmax(outputs_benign, dim=-1)
#         #print (outputs_benign[0])


#         loss_benign =  criterion_train(outputs_benign,label)

#         #print("loss computed")
#         loss_benign.backward()
#         #print("loss BP")
#         optimizer_train.step()
#         #sgd_update(net.parameters())

#         #if i%2000==0:
#         #print (loss_benign.item())
#         #torch.save(net.state_dict(),'./LFW_net.pth')  

#         #if  iter_%50==0:
#         #    print ('attack',iter_)
       
        
#         print ('fininshed training')
#         break
###############################   testing
    
#         total = len(y_test)
#         acc =0.0
#         for ct in range(total):
#             testing_data = tt(testing[ct][0].cpu())
#             testing_data1 = tp(testing_data).to(device)
#             testing_data2 = testing_data1.view(1, *testing_data1.size())
#             y_pred = net(testing_data2)
#             predicted = torch.argmax(y_pred)

#             if predicted == y_test[ct]:
#                 acc=acc+1
#         accuracy = acc / total
#         print (accuracy)
#         print ('fininshed testing')



In [ ]:
from pytorch_msssim import ssim


for epoch in range(1):

    for iter_,data in enumerate(trainloader,1):
   
        ######### honest partipant #########
        img_index = 30   #use img_index
        dst_pil = tt(dst_tensor[img_index][0].cpu())   #use img_index

        gt_data = transform(dst_pil).to(device)
        gt_data = torch.unsqueeze(gt_data,0)

        gt_label = dst_tensor[img_index][1].long().to(device) #use img_index
        gt_label = gt_label.view(1, )
        gt_onehot_label = label_to_onehot(gt_label, num_classes=3)

        plt.imshow(dst_pil)
        #plt.savefig("./original/index_%s_label_%s"%(img_index,gt_label.item()))



        batch =32  #
        for bat in range(batch-1):
            dst_pil = tt(dst_tensor[img_index+1+bat][0].cpu())   #use img_index
            tmp = torch.unsqueeze(transform(dst_pil).to(device),0)
            #print(tmp.shape)
            gt_data = torch.cat((gt_data,tmp),0)

            gt_label_tmp = dst_tensor[img_index+1+bat][1].long().to(device) #use img_index
            gt_label_tmp = gt_label_tmp.view(1, )
            gt_label = torch.cat((gt_label,gt_label_tmp),0)
            gt_onehot_label = torch.cat((gt_onehot_label,label_to_onehot(gt_label_tmp, num_classes=3)),0)

            plt.imshow(dst_pil)
            #plt.savefig("./original/index_%s_label_%s"%(bat+1,gt_label_tmp.item()))

            #plt.title("Ground truth image")
            #print("GT label is %d." % gt_label.item(), "\nOnehot label is %d." % torch.argmax(gt_onehot_label, dim=-1).item())


        gt_label = torch.reshape(gt_label,(-1,1))    
        print ("Image Shape = ",gt_data.shape)
        print ("Label shape = ",gt_label.shape)
        print ("Onehot Label shape = ",gt_onehot_label.shape)


        # compute original gradient 
        dy_dx = []
        original_dy_dx=[]
        original_pred = []
        for item in range(batch):
            gt_data_single = torch.unsqueeze(gt_data[item],0)
            out = net(gt_data_single)
            #y = criterion(out, gt_onehot_label[item])
            y = criterion(out, gt_label[item])
            dy_dx = torch.autograd.grad(y, net.parameters(),retain_graph=True)
            original_dy_dx_tmp = list((_.detach().clone() for _ in dy_dx))
            original_dy_dx.append(original_dy_dx_tmp)
            out_tmp = out.detach().clone()
            original_pred.append(out_tmp)

            #dy_dx.append(torch.autograd.grad(y, net.parameters()))





        # share the gradients with other clients
        #original_dy_dx = list((_.detach().clone() for _ in dy_dx))


        # generate dummy data and label
        import time


        for item in range(1):
            start = time.process_time()
            for rd in range(1):

                torch.manual_seed(100*rd)
        
                pat_1 = torch.rand([3,16,16])
                pat_2 = torch.cat((pat_1,pat_1),dim=1)
                pat_4 = torch.cat((pat_2,pat_2),dim=2)
                dummy_data = torch.unsqueeze(pat_4,dim=0).to(device).requires_grad_(True)     

                dummy_unsqueeze=torch.unsqueeze(gt_onehot_label[item],dim=0)

                dummy_label = torch.randn(dummy_unsqueeze.size()).to(device).requires_grad_(True)
                label_pred=torch.argmin(torch.sum(original_dy_dx[item][-2], dim=-1), 
                                        dim=-1).detach().reshape((1,)).requires_grad_(False)
                #print (original_dy_dx[item][-1].shape)
                #print (original_dy_dx[item][-1].argmin())

                #print (torch.sum(original_dy_dx[item][-2], dim=-1).argmin())

                plt.imshow(tt(dummy_data[0].cpu()))
                plt.title("Dummy data")
                #plt.savefig("./random_seed/index_%s_rand_seed_%s_label_%s"%(item,rd,torch.argmax(dummy_label, dim=-1).item()))

                plt.clf()
                print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
                print("stolen label is %d." % label_pred.item())


                #optimizer = torch.optim.LBFGS([dummy_data,dummy_label])
                optimizer = torch.optim.LBFGS([dummy_data,])
                #optimizer = torch.optim.AdamW([dummy_data,],lr=0.01)


                history = []
                history_batch = []
                history_grad = []
                
                percept_dis = np.zeros(500)
                recover_dis = np.zeros(500)
                for iters in range(500):
                    
                    percept_dis[iters]=ssim(dummy_data,torch.unsqueeze(gt_data[item],dim=0),data_range=0).item()
                    #recover_dis[iters]=torch.dist(dummy_data,torch.unsqueeze(gt_data[item],dim=0),2).item()
                    recover_dis[iters]= F.mse_loss(dummy_data,torch.unsqueeze(gt_data[item],dim=0)).item()
                    
                    history.append(tt(dummy_data[0].cpu()))

                    def closure():
                        optimizer.zero_grad()

                        pred = net(dummy_data) 
                        #dummy_onehot_label = F.softmax(dummy_label, dim=-1).long()

                        #dummy_loss = criterion(pred, dummy_onehot_label) # TODO: fix the gt_label to dummy_label in both code and slides.
                        #print (pred)
                        #print (label_pred)

                        dummy_loss = criterion(pred, label_pred)
                        dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)
                        #dummy_dy_dp = torch.autograd.grad(dummy_loss, dummy_data, create_graph=True)
                        #print (dummy_dy_dp[0].shape)  

                        grad_diff = 0
                        grad_count = 0
                        #count =0
                        for gx, gy in zip(dummy_dy_dx, original_dy_dx[item]): # TODO: fix the variablas here

                            #if iters==500 or iters== 1200:
                            #print (gx[0])
                            #    print ('hahaha')
                            #print (gy[0])
                            lasso = torch.norm(dummy_data,p=1)
                            ridge = torch.norm(dummy_data,p=2)
                            grad_diff += ((gx - gy) ** 2).sum() #+ 0.0*lasso +0.01*ridge 

                            #print (gx.shape)

                            grad_count += gx.nelement()


                            #if count == 9:
                            #    break
                            #count=count+1
                        # grad_diff = grad_diff / grad_count * 1000

                        #grad_diff += ((original_pred[item]-pred)**2).sum()




                        grad_diff.backward()
                        #print (count)

                        #print (dummy_dy_dx)
                        #print (original_dy_dx)


                        return grad_diff



                    optimizer.step(closure)
                    if iters % 5 == 0: 
                        current_loss = closure()
                        #if iters == 0: 
                        #print ("%.8f" % current_loss.item())
                        #print(iters, "%.8f" % current_loss.item())

                #     for bat in range(batch-1):
                #         history_batch.append(tt(dummy_data[bat].cpu()))

                plt.figure(figsize=(30, 20))
                for i in range(100):
                    plt.subplot(10, 10, i + 1)
                    plt.imshow(history[i * 5])
                    plt.title("iter=%d" % (i * 5))
                    plt.axis('off')
                #print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())
                
                #np.savetxt('lfw_ssim_%s'%iter_,percept_dis,fmt="%4f")
                #np.savetxt('lfw_mse_%s'%iter_,recover_dis,fmt="%4f")
                #plt.savefig("./attack_image/index_%s_iter_%s_label_%s"%(img_index,iter_,torch.argmax(dummy_label, dim=-1).item()))
                
                plt.clf()
            duration = time.process_time()-start
            #print ("Running time is %.4f." %(duration/10.0) )
            print ("Duration = ", duration)
            
            
            #if epoch>=1:
        #if i==1:
            #break
        #print (iter_)
        inputs,label = data

        inputs,label =  Variable(inputs),Variable(label) 

        optimizer_train.zero_grad()


        outputs_benign=net(inputs)
        #outputs_benign = F.softmax(outputs_benign, dim=-1)
        #print (outputs_benign[0])


        loss_benign =  criterion_train(outputs_benign,label)

        #print("loss computed")
        loss_benign.backward()
        #print("loss BP")
        optimizer_train.step()

        #if i%2000==0:
        print ("Loss Benign = ",loss_benign.item())
        #torch.save(net.state_dict(),'./LFW_net.pth')  

  
        print ('fininshed training')
        total = len(y_test)
        acc =0.0
        for ct in range(total):
            testing_data = tt(testing[ct][0].cpu())
            testing_data1 = transform(testing_data).to(device)
            testing_data2 = testing_data1.view(1, *testing_data1.size())
            y_pred = net(testing_data2)
            predicted = torch.argmax(y_pred)

            if predicted == y_test[ct]:
                acc=acc+1
        accuracy = acc / total
        print ("accuracy = ",accuracy)
        print ('fininshed testing')
        print("ssim_random2 = ", percept_dis)
        print("mse_random2 = ", recover_dis)


        

In [ ]:
plt.figure(figsize=(12, 8))
for i in range(60):
  plt.subplot(6, 10, i + 1)
  plt.imshow(history[i * 5])
  plt.title("iter=%d" % (i * 5))
  plt.axis('off')
print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())

In [ ]:
plt.imshow(history[295])

In [ ]:
import torch
import torchvision
import torchvision.transforms.functional as F

# Load the two images (as PIL Images or tensors)
img1 = X_train[21]
img2 = history[295]

# Convert the images to tensors and reshape to (batch_size, channels, height, width)
img1_tensor = F.to_tensor(img1).unsqueeze(0)
img2_tensor = F.to_tensor(img2).unsqueeze(0)

# Calculate SSIM between the two images
ssim_value = ssim(img1_tensor, img2_tensor, data_range=1.0, size_average=True)

print(f"SSIM between the two images: {ssim_value:.4f}")

In [ ]:
import torch.nn.functional as F
mse = F.mse_loss(img1_tensor, img2_tensor)
print(f"SSIM: {mse:.4f}")